In [1]:
# 목표 예측 범위 안에 넣기
# data * std + mean


In [2]:
%matplotlib widget

In [3]:
import pandas as pd
import numpy as np
from glob import glob
import os
import datetime
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Input, Concatenate, Dot, Add, ReLU, Activation
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow import keras

In [4]:
folder = 'data'
# file_names = [['가평_2016.xlsx','가평_2017.xlsx','가평_2018.xlsx', '가평_2019.xlsx'], 
#               ['의암호_2016.xlsx','의암호_2017.xlsx','의암호_2018.xlsx', '의암호_2019.xlsx'],
#               ['서상_2016.xlsx','서상_2017.xlsx','서상_2018.xlsx', '서상_2019.xlsx']]

file_names = [['가평_2016.xlsx','가평_2017.xlsx','가평_2018.xlsx', '가평_2019.xlsx'], 
              ['의암호_2016.xlsx','의암호_2017.xlsx','의암호_2018.xlsx', '의암호_2019.xlsx'],
             ]
day = 24*60*60
year = (365.2425)*day

df_full = []
df = []

for loc in range(len(file_names)):
    
    df_loc = []
    for y in range(len(file_names[loc])):
        path = os.path.join(folder, file_names[loc][y])
        print(file_names[loc][y])
        df_loc.append(pd.read_excel(path))
    df_full.append(pd.concat(df_loc))
    df.append(df_full[loc].iloc[:, 2:])
#     df.append(df_full[loc].iloc[:, [2,3,4,5,6,7,10]])
#     df.append(df_full[loc].iloc[:, 2:11])
    date_time = pd.to_datetime(df_full[loc].iloc[:, 0], format='%Y.%m.%d %H:%M', utc=True)
    timestamp_s = date_time.map(datetime.datetime.timestamp)
    df[loc]['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
    df[loc]['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
    df[loc]['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
    df[loc]['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))
    df[loc] = df[loc].reset_index(drop=True)
        

가평_2016.xlsx
가평_2017.xlsx
가평_2018.xlsx
가평_2019.xlsx
의암호_2016.xlsx
의암호_2017.xlsx
의암호_2018.xlsx
의암호_2019.xlsx


In [5]:
df[0]

,수온,수소이온농도,전기전도도,용존산소,탁도,총유기탄소,총질소,총인,클로로필-a,미생물_독성지수,Day sin,Day cos,Year sin,Year cos
0,4.5,7.5,126.0,10.8,0.6,1.4,2.398,0.006,5.7,0.5,-1.407890e-12,1.000000,-0.002666,0.999996
1,4.3,7.5,126.0,10.8,0.5,1.4,2.490,0.003,5.4,0.4,2.588190e-01,0.965926,-0.001950,0.999998
2,4.4,7.5,126.0,10.8,0.5,1.4,2.545,0.003,5.4,0.5,5.000000e-01,0.866025,-0.001233,0.999999
3,4.4,7.5,126.0,10.8,0.6,1.4,2.482,0.002,5.4,4.7,7.071068e-01,0.707107,-0.000516,1.000000
4,4.4,7.5,126.0,10.8,0.6,1.4,2.453,0.003,5.4,-2.5,8.660254e-01,0.500000,0.000201,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,3.6,7.1,115.0,12.6,1.1,1.8,1.718,0.002,22.7,0.1,-9.659258e-01,0.258819,-0.005734,0.999984
35060,3.5,7.1,115.0,12.6,1.1,1.7,1.726,0.001,21.9,0.5,-8.660254e-01,0.500000,-0.005017,0.999987
35061,3.4,7.1,115.0,12.6,1.1,1.7,1.738,0.002,21.3,0.9,-7.071068e-01,0.707107,-0.004301,0.999991
35062,3.4,7.1,115.0,12.6,1.1,1.7,1.723,0.002,21.5,0.7,-5.000000e-01,0.866025,-0.003584,0.999994


In [6]:
df[0]

,수온,수소이온농도,전기전도도,용존산소,탁도,총유기탄소,총질소,총인,클로로필-a,미생물_독성지수,Day sin,Day cos,Year sin,Year cos
0,4.5,7.5,126.0,10.8,0.6,1.4,2.398,0.006,5.7,0.5,-1.407890e-12,1.000000,-0.002666,0.999996
1,4.3,7.5,126.0,10.8,0.5,1.4,2.490,0.003,5.4,0.4,2.588190e-01,0.965926,-0.001950,0.999998
2,4.4,7.5,126.0,10.8,0.5,1.4,2.545,0.003,5.4,0.5,5.000000e-01,0.866025,-0.001233,0.999999
3,4.4,7.5,126.0,10.8,0.6,1.4,2.482,0.002,5.4,4.7,7.071068e-01,0.707107,-0.000516,1.000000
4,4.4,7.5,126.0,10.8,0.6,1.4,2.453,0.003,5.4,-2.5,8.660254e-01,0.500000,0.000201,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,3.6,7.1,115.0,12.6,1.1,1.8,1.718,0.002,22.7,0.1,-9.659258e-01,0.258819,-0.005734,0.999984
35060,3.5,7.1,115.0,12.6,1.1,1.7,1.726,0.001,21.9,0.5,-8.660254e-01,0.500000,-0.005017,0.999987
35061,3.4,7.1,115.0,12.6,1.1,1.7,1.738,0.002,21.3,0.9,-7.071068e-01,0.707107,-0.004301,0.999991
35062,3.4,7.1,115.0,12.6,1.1,1.7,1.723,0.002,21.5,0.7,-5.000000e-01,0.866025,-0.003584,0.999994


In [7]:
# normalize data

df_all = pd.concat(df)
df_all

train_mean = df_all.mean()
train_std = df_all.std()
for i in range(len(file_names)):
    df[i] = (df[i]-train_mean)/train_std

In [8]:
train_mean, train_std


(수온          1.442927e+01
 수소이온농도      7.338229e+00
 전기전도도       1.067481e+02
 용존산소        1.051423e+01
 탁도          2.159194e+00
 총유기탄소       1.680518e+00
 총질소         1.717054e+00
 총인          7.725232e-03
 클로로필-a      1.064802e+01
 미생물_독성지수   -4.499445e-02
 Day sin     6.037379e-15
 Day cos     2.747517e-15
 Year sin   -5.681266e-08
 Year cos    2.053380e-05
 dtype: float64,
 수온           7.397900
 수소이온농도       0.485797
 전기전도도       13.159411
 용존산소         1.806036
 탁도           5.437522
 총유기탄소        0.368691
 총질소          0.382042
 총인           0.007932
 클로로필-a      10.570897
 미생물_독성지수     1.615261
 Day sin      0.707112
 Day cos      0.707112
 Year sin     0.707105
 Year cos     0.707119
 dtype: float64)

In [9]:
df[0]

,수온,수소이온농도,전기전도도,용존산소,탁도,총유기탄소,총질소,총인,클로로필-a,미생물_독성지수,Day sin,Day cos,Year sin,Year cos
0,-1.342174,0.333002,1.462974,0.158231,-0.286747,-0.760849,1.782384,-0.21750,-0.468079,0.337403,-1.999582e-12,1.414203,-0.003771,1.414155
1,-1.369209,0.333002,1.462974,0.158231,-0.305138,-0.760849,2.023195,-0.59571,-0.496459,0.275494,3.660228e-01,1.366016,-0.002757,1.414157
2,-1.355692,0.333002,1.462974,0.158231,-0.305138,-0.760849,2.167159,-0.59571,-0.496459,0.337403,7.071017e-01,1.224736,-0.001743,1.414159
3,-1.355692,0.333002,1.462974,0.158231,-0.286747,-0.760849,2.002255,-0.72178,-0.496459,2.937603,9.999929e-01,0.999993,-0.000730,1.414160
4,-1.355692,0.333002,1.462974,0.158231,-0.286747,-0.760849,1.926347,-0.59571,-0.496459,-1.519882,1.224736e+00,0.707102,0.000284,1.414160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,-1.463830,-0.490387,0.627070,1.154889,-0.194794,0.324071,0.002475,-0.72178,1.140110,0.089765,-1.366016e+00,0.366023,-0.008109,1.414137
35060,-1.477348,-0.490387,0.627070,1.154889,-0.194794,0.052841,0.023416,-0.84785,1.064430,0.337403,-1.224736e+00,0.707102,-0.007096,1.414142
35061,-1.490865,-0.490387,0.627070,1.154889,-0.194794,0.052841,0.054826,-0.72178,1.007671,0.585041,-9.999929e-01,0.999993,-0.006082,1.414147
35062,-1.490865,-0.490387,0.627070,1.154889,-0.194794,0.052841,0.015563,-0.72178,1.026590,0.461222,-7.071017e-01,1.224736,-0.005068,1.414151


In [10]:
train_df = df[0]
val_df = df[0]
test_df = df[0]

In [11]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
            #train_df=None, val_df=None, test_df=None,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [12]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [13]:
import matplotlib
import matplotlib.font_manager as fm
fm.get_fontconfig_fonts()
font_location = '/usr/share/fonts/truetype/nanum/NanumGothicCoding.ttf'
#font_location = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
# font_location = 'C:/Windows/Fonts/NanumGothic.ttf' # For Windows
fprop = fm.FontProperties(fname=font_location)

In [14]:
def plot(self, model=None, plot_col='T (degC)', max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot = plot

In [15]:
# not used
# original make_dataset code
def make_dataset(self, data):
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.preprocessing.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      batch_size=32,)

  ds = ds.map(self.split_window)

  return ds

#WindowGenerator.make_dataset = make_dataset

In [16]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

In [17]:
def sample_batch_index(total, batch_size):
    '''Sample index of the mini-batch.

    Args:
        - total: total number of samples
        - batch_size: batch size

    Returns:
        - batch_idx: batch index
    '''
    total_idx = np.random.permutation(total)
    batch_idx = total_idx[:batch_size]
    return batch_idx

In [18]:
def binary_sampler(p, shape):
  '''Sample binary random variables.
  
  Args:
    - p: probability of 1
    - shape: matrix shape
    
  Returns:
    - binary_random_matrix: generated binary random matrix.
  '''
  unif_random_matrix = np.random.uniform(0., 1., size = shape)
  binary_random_matrix = 1*(unif_random_matrix < p)
  return binary_random_matrix

In [19]:
def uniform_sampler(low, high, shape):
  '''Sample uniform random variables.
  
  Args:
    - low: low limit
    - high: high limit
    - rows: the number of rows
    - cols: the number of columns
    
  Returns:
    - uniform_random_matrix: generated uniform random matrix.
  '''
  return np.random.uniform(low, high, size = shape)

In [20]:
def normalization (data, parameters=None):
  '''Normalize data in [0, 1] range.
  
  Args:
    - data: original data
  
  Returns:
    - norm_data: normalized data
    - norm_parameters: min_val, max_val for each feature for renormalization
  '''

  # Parameters
  _, dim = data.shape
  norm_data = data.copy()

  if parameters is None:

    # MixMax normalization
    min_val = np.zeros(dim)
    max_val = np.zeros(dim)
   
    # For each dimension
    for i in range(dim):
      min_val[i] = np.nanmin(norm_data[:,i])
      norm_data[:,i] = norm_data[:,i] - np.nanmin(norm_data[:,i])
      max_val[i] = np.nanmax(norm_data[:,i])
      norm_data[:,i] = norm_data[:,i] / (np.nanmax(norm_data[:,i]) + 1e-6)

    # Return norm_parameters for renormalization
    norm_parameters = {'min_val': min_val,
                       'max_val': max_val}
  else:
    min_val = parameters['min_val']
    max_val = parameters['max_val']

    # For each dimension
    for i in range(dim):
      norm_data[:,i] = norm_data[:,i] - min_val[i]
      norm_data[:,i] = norm_data[:,i] / (max_val[i] + 1e-6)

    norm_parameters = parameters

  return norm_data, norm_parameters

In [21]:
class MissData(object):
    def __init__(self, load_dir=None):
        if load_dir:
            self.missarr = np.load(os.path.join(load_dir, 'miss.npy'))
            self.idxarr = np.load(os.path.join(load_dir, 'idx.npy'))
            
    def make_missdata(self, data_x, missrate=0.2):
        data = data_x.copy()
        rows, cols = data_x.shape
        total_no = rows*cols
        total_miss_no = np.round(total_no*missrate).astype(int)
        total_idx = self.idxarr.shape[0]
        idxarr = self.idxarr
        missarr = self.missarr
        #print(total_miss_no)
        miss_no = 0
        cum_no = self.idxarr[:,3:4]
        cum_no = cum_no.reshape((total_idx))
        cum_sum = np.max(cum_no)
        #print(cum_no)
        #print(total_idx)
        while True:
            loc_count = np.around(np.random.random()*cum_sum)
            #print('loc_count =', loc_count)
            idx = len(cum_no[cum_no <= loc_count])-1
            #print(cum_no[cum_no <= loc_count])
            #print('idx =', idx)
            startnan = idxarr[idx][0]
            nanlen = idxarr[idx][2]
            loc = np.around(np.random.random()*(rows-nanlen)).astype(int)
            #print('loc =', loc)
            #print(loc_count, idx)
            #print(idxarr[idx])
            #data_copy = data[loc:loc+nanlen].copy()
            data_copy = data[loc:loc+nanlen]
            #print('startnan=', startnan)
            #isnan = missarr[startnan:startnan+nanlen].copy()
            isnan = missarr[startnan:startnan+nanlen]
            #print('isnan =',isnan)
            miss_no += idxarr[idx][1]
            if (miss_no > total_miss_no):
                break
            data_copy[isnan==1] = np.nan
            data[loc:loc+nanlen] = data_copy
        #print('miss_data =', data)
        return data
    
    def save(data, max_tseq, save_dir='save'):
        no, dim = data.shape
        #print((no, dim))
        isnan = np.isnan(data).astype(int)
        isany = np.any(isnan, axis=1).astype(int)
        shifted = np.roll(isany, 1)
        shifted[0] = 1
        #print(isnan)
        #print(isany.astype(int))
        #print(shifted)
        startnan = ((isany == 1) & (shifted ==0)).astype(int)
        #print(startnan)
        group = startnan.cumsum()
        group = group*isany
        #print(group)
        n = np.max(group)
        #print(n)
        missarr = None
        cum_no = 0
        rowidx = 0
        for i in range(1, n+1):
            g = (group == i).astype(int)
            i = np.argmax(g)
            rows = g.sum()
            #print(len)
            #print(i)
            #print(type(missarr))
            if rows <= max_tseq:
                nanseq = isnan[i:i+rows, :]
                no = np.sum(nanseq)
                #print(no)
                if missarr is None:
                    missarr = nanseq
                    idxarr = np.array([[rowidx, no, rows, cum_no]])
                else:
                    missarr = np.concatenate((missarr, nanseq))
                    idxarr = np.concatenate((idxarr, [[rowidx, no, rows, cum_no]]), axis=0)
                cum_no += no
                rowidx += rows

        #print(idxarr)
        miss_npy_file = os.path.join(save_dir, 'miss.npy')
        idx_npy_file = os.path.join(save_dir, 'idx.npy')
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        np.save(miss_npy_file, missarr)
        np.save(idx_npy_file, idxarr)
        print('miss_data file saved')

In [22]:
norm_df = pd.concat(df,axis=0)
norm_data = norm_df.to_numpy()
MissData.save(norm_data, max_tseq = 24)

miss_data file saved


In [23]:
def interpolate(np_data, max_gap=3):
    #n = np_data.shape[1]
    data = pd.DataFrame(np_data)
    #data[0][0] = np.nan
    #data[0][1] = np.nan
    #data[0][2] = np.nan
    #data[data.columns[0]][0] = np.nan
    #data[data.columns[0]][1] = np.nan
    #data[data.columns[0]][2] = np.nan
    
    # create mask
    mask = data.copy()
    grp = ((mask.notnull() != mask.shift().notnull()).cumsum())
    grp['ones'] = 1
    for i in data.columns:
        mask[i] = (grp.groupby(i)['ones'].transform('count') < max_gap) | data[i].notnull()
    data = data.interpolate(method='polynomial', order=5, limit=max_gap, axis=0).bfill()[mask]
    return data.to_numpy()
    #return data
    
#filled_data = interpolate(norm_data, max_gap=3)
#np.arange(0, 5, dtype=int)
#['%d'%val for val in range(0,5)]

In [24]:
class GainDataGenerator(keras.utils.Sequence):
    'Generates data for GAIN'
    def __init__(self,
                 data_list,
                 batch_size=32,
                 input_width=24*3,
                 label_width=24*3,
                 shift=0,
                 fill_no=4,
                 miss_rate=0.2,
                 hint_rate=0.9,
                 normalize=True,
                 miss_pattern=None,
                 alpha=100.):
        'Initialization'
        window_size = input_width
        
        # interpollation
        filled_data = []
        for data in data_list:
            data = interpolate(data, max_gap=fill_no)
            filled_data.append(data)
            
        data_list = filled_data
        
        # whole data
        self.data = np.concatenate(data_list)

        # TO-DO
        
        # pre calculation for  sequence data
        last_cum = 0
        cums = []
        for data in data_list:
            isnan = np.isnan(data)
            isany = np.any(isnan, axis=1)
            #shift same as pd.shift(isany, fill_value=True)
            shifted = np.roll(isany, 1)
            shifted[0] = True # set to nan
            
            start_seq = ((isany == False) & (shifted == True)).astype(int)
            cum = start_seq.cumsum()
            cum += last_cum
            last_cum = np.max(cum)
            cum[isany] = 0
            cums.append(cum)
            
        
        # normlize for spam
        if normalize:
            self.data, norm_param = normalization(self.data)
        #print(norm_param)
        
        # Define mask matrix
        if miss_pattern is None:
            self.data_m = binary_sampler(1-miss_rate, self.data.shape)
        else:
            #MissData.save(self.data, max_tseq = 12)
            self.miss = MissData(load_dir='save')
            self.miss_rate = miss_rate
            miss_data = self.miss.make_missdata(self.data, self.miss_rate)
            self.data_m = 1. - np.isnan(miss_data).astype(float)
            
            self.data_m_rand = binary_sampler(1-(miss_rate/10.), self.data.shape)
            self.data_m[self.data_m_rand==0.] = 0.
        self.miss_pattern = miss_pattern
        
        # sequence data
        self.ids = np.concatenate(cums)
        data_idx = np.empty((0), dtype=int)
        for i in range(1, last_cum+1):
            seq_len = (self.ids == i).sum()
            start_id = np.argmax(self.ids == i)
            # possible data number in seqeunce
            time_len = seq_len - window_size + 1
            start_ids = np.arange(start_id, start_id+time_len)
            data_idx = np.append(data_idx, start_ids)
            
        # start index set for sequence data
        self.data_idx = data_idx
        self.input_width = input_width
        self.no = len(data_idx)
        
        #print('self.no = ', self.no)
        
        self.batch_size = batch_size
        
        # random shuffling  index
        self.batch_idx = sample_batch_index(self.no, self.no)
        self.batch_id = 0
        self.shape = (batch_size,self.input_width)+self.data.shape[1:]
        #self.hint_rate = hint_rate
            
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        # Sample batch
        x = np.empty((0, self.input_width, self.data.shape[1]))
        #m = np.empty((0, self.input_width, self.data.shape[1]))
        #h = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.input_width, self.data.shape[1]))
        #print(x.shape)
        #print(self.data.shape)
        #print(self.input_width)
        #self.batch_idx = sample_batch_index(self.no, self.batch_size)
        for cnt in range(0, self.batch_size):
            i = self.batch_idx[self.batch_id]
            self.batch_id += 1
            #self.batch_id %= self.batch_size
            self.batch_id %= self.no
            if self.miss_pattern and (self.batch_id == 0):
                self.batch_idx = sample_batch_index(self.no, self.no)
                miss_data = self.miss.make_missdata(self.data, self.miss_rate)
                self.data_m = 1. - np.isnan(miss_data).astype(float)
                self.data_m_rand = binary_sampler(1-self.miss_rate/10., self.data.shape)
                self.data_m[self.data_m_rand==0.] = 0.
            idx1 = self.data_idx[i]
            idx2 = self.data_idx[i]+self.input_width
            #print(idx1, idx2)
        
            Y_mb = self.data[idx1:idx2].copy()
            X_mb = Y_mb.copy()
            M_mb = self.data_m[idx1:idx2]
            Z_mb = uniform_sampler(0, 0.01, shape=X_mb.shape)
            X_mb = M_mb*X_mb + (1-M_mb)*Z_mb
            #H_mb_temp = binary_sampler(self.hint_rate, shape=X_mb.shape)
            #H_mb = M_mb * H_mb_temp
            X_mb[M_mb == 0] = np.nan
            Y_mb[M_mb == 1] = np.nan
            x = np.append(x, [X_mb], axis=0)
            #m = np.append(m, [M_mb], axis=0)
            #h = np.append(h, [H_mb], axis=0)
            y = np.append(y, [Y_mb], axis=0)
            
        #return [x, m, h], y
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

dgen = GainDataGenerator(df)

In [25]:
class GAIN(keras.Model):
    def __init__(self, shape, alpha=100., load=False, hint_rate=0.9, gen_sigmoid=True, **kwargs):
        super(GAIN, self).__init__(**kwargs)
        self.shape = shape
        self.dim = np.prod(shape).astype(int)
        self.h_dim = self.dim
        self.gen_sigmoid = gen_sigmoid
        self.build_generator()
        self.build_discriminator()
        self.hint_rate = hint_rate
        self.alpha = alpha
        self.generator_optimizer = Adam()
        self.discriminator_optimizer = Adam()

    ## GAIN models
    def build_generator(self):
        last_activation = 'sigmoid' if self.gen_sigmoid else None
        xavier_initializer = tf.keras.initializers.GlorotNormal()

        shape = self.shape
        #x = Input(shape=(self.dim,), name='generator_input_x')
        #m = Input(shape=(self.dim,), name='generator_input_m')
        x = Input(shape=shape, name='generator_input_x')
        m = Input(shape=shape, name='generator_input_m')
        
        x_f = Flatten()(x)
        m_f = Flatten()(m)

        a = Concatenate()([x_f, m_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.dim, activation=last_activation, kernel_initializer=xavier_initializer)(a)
        G_prob = keras.layers.Reshape(shape)(a)
        self.generator = keras.models.Model([x, m], G_prob, name='generator')

    def build_discriminator(self):
        xavier_initializer = tf.keras.initializers.GlorotNormal()
        shape = self.shape

        #x = Input(shape=(self.dim,), name='discriminator_input_x')
        #h = Input(shape=(self.dim,), name='discriminator_input_h')
        x = Input(shape=shape, name='discriminator_input_x')
        h = Input(shape=shape, name='discriminator_input_h')
        
        x_f = Flatten()(x)
        h_f = Flatten()(h)

        a = Concatenate()([x_f, h_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.dim, activation='sigmoid', kernel_initializer=xavier_initializer)(a)
        D_prob = keras.layers.Reshape(shape)(a)
        self.discriminator = keras.models.Model([x, h], D_prob, name='discriminator')
        
    def call(self, inputs):
        if isinstance(inputs, tuple):
            inputs = inputs[0]
        shape = inputs.shape
        dims = np.prod(shape[1:])
        input_width = shape[1]
        #print('inputs.shape=',inputs.shape)
        x = inputs
        #x = x.reshape((n, -1))
        #print('dims=',dims)
        #x = keras.layers.Reshape((dims,))(x)
        #x = keras.layers.Reshape(tf.TensorShape((self.dim,)))(x)
        #print('x =', x)
        #print('x.shape = ', x.shape)
        #x = keras.layers.Reshape(tf.TensorShape([57]))(x)
        
        isnan = tf.math.is_nan(x)
        #m = 1.- keras.backend.cast(isnan, dtype=tf.float32)
        m = tf.where(isnan, 0., 1.)
        z = keras.backend.random_uniform(shape=tf.shape(x), minval=0.0, maxval=0.01)
        x = tf.where(isnan, z, x)
        #z = uniform_sampler(0, 0.01, shape=x.shape)
        #z = tf.keras.backend.random_uniform(shape=x.shape, minval=0.0, maxval=0.01)
        imputed_data = self.generator([x, m], training=False)
        #imputed_data = m*x + (1-m)*imputed_data
        #imputed_data = tf.where(isnan, imputed_data, np.nan)
        imputed_data = tf.where(isnan, imputed_data, x)
        #imputed_data = keras.layers.Reshape(shape[1:])(imputed_data)
        #print('imputed_data.shape = ', imputed_data.shape)
        
        return imputed_data
    
    def D_loss(M, D_prob):
        ## GAIN loss
        return -tf.reduce_mean(M * tf.keras.backend.log(D_prob + 1e-8) \
                         + (1-M) * tf.keras.backend.log(1. - D_prob + 1e-8))
    
    def G_loss(self, M, D_prob, X, G_sample):
        G_loss_temp = -tf.reduce_mean((1-M) * tf.keras.backend.log(D_prob + 1e-8))
        MSE_loss = tf.reduce_mean((M * X - M * G_sample)**2) / (tf.reduce_mean(M) + 1e-8)
        #G_loss_temp = GAIN.G_loss_bincross(M, D_prob)
        #MSE_loss = GAIN.MSE_loss(M, X, G_sample)
        G_loss = G_loss_temp + self.alpha * MSE_loss
        return G_loss
        
    def RMSE_loss(y_true, y_pred):
        isnan = tf.math.is_nan(y_true)
        M = tf.where(isnan, 1., 0.)
        return tf.sqrt(tf.reduce_sum(tf.where(isnan, 0., y_pred-y_true)**2)/tf.reduce_sum(1-M))
    
    def train_step(self, data):
        #[x, m, h], y = data
        x, y = data
        #X = keras.layers.Reshape((self.dim,), input_shape=self.shape)(x)
        #Y = keras.layers.Reshape((self.dim,), input_shape=self.shape)(y)
        #X = keras.layers.Flatten()(x)
        #Y = keras.layers.Flatten()(y)
        X = x
        Y = y
        #X = tf.reshape(x, shape=(x.shape[0], -1))
        #Y = tf.reshape(y, shape=(x.shape[0], -1))
        isnan = tf.math.is_nan(X)
        #M = 1 - keras.backend.cast(isnan, dtype=tf.float32)
        M = tf.where(isnan, 0., 1.)
        Z = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=0.01)
        #H_temp = binary_sampler(self.hint_rate, shape=X.shape)
        H_rand = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=1.)
        #H_temp = 1*keras.backend.cast((H_rand < self.hint_rate), dtype=tf.float32)
        H_temp = tf.where(H_rand < self.hint_rate, 1., 0.)
        
        H = M * H_temp
        #X = M * X + (1-M) * Z
        X = tf.where(isnan, Z, X)
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            G_sample = self.generator([X, M], training=True)

            # Combine with observed data
            #Hat_X = tf.where(isnan, G_sample, X)
            Hat_X = X * M + G_sample * (1-M)
            D_prob = self.discriminator([Hat_X, H], training=True)
            gen_loss = self.G_loss(M, D_prob, X, G_sample)
            disc_loss = tf.keras.backend.mean(tf.keras.losses.binary_crossentropy(M, D_prob))
            #disc_loss = GAIN.D_loss(M, D_prob)
            #disc_loss = GAIN.D_loss(M, D_prob)

        gradients_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

        self.generator_optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))
        self.discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))
        
        rmse = tf.sqrt(tf.reduce_sum(tf.where(isnan, G_sample - Y, 0.)**2)/tf.reduce_sum(1-M))
        return {
                 'gen_loss':     gen_loss,
                 'disc_loss':    disc_loss,
                 'rmse':         rmse,
               }
    
    def save(self, save_dir='save'):
        if not os.path.exists(save_dir):
          os.makedirs(save_dir)
        disc_savefile = os.path.join(save_dir, 'discriminator.h5')
        gen_savefile = os.path.join(save_dir, 'generator.h5')
        self.discriminator.save_weights(disc_savefile)
        self.generator.save_weights(gen_savefile)

    def load(self, save_dir='save'):
        disc_savefile = os.path.join(save_dir, 'discriminator.h5')
        gen_savefile = os.path.join(save_dir, 'generator.h5')
        try:
          self.discriminator.load_weights(disc_savefile)
          self.generator.load_weights(gen_savefile)
          print('model weights loaded')
        except:
          print('model loadinng error')

In [26]:
def make_dataset_gain(self, data):
  dg = GainDataGenerator(
      df,
      input_width = self.input_width,
      label_width = self.label_width,
      batch_size = 128,
      normalize = False,
      miss_pattern = True,
      miss_rate = 0.15,
      fill_no = 3,
  )
  self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.shape,
        dg.shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  return ds

WindowGenerator.make_dataset = make_dataset_gain

In [27]:
train_df = df_all
val_df = df_all
test_df = df_all

In [28]:
wide_window = WindowGenerator(
    input_width=24*5, label_width=24*5, shift=0,
    #label_columns=['T (degC)']
)

wide_window

Total window size: 120
Input indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]
Label indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 

In [29]:
wide_window.example[0].shape

TensorShape([128, 120, 14])

In [30]:
df[0]

,수온,수소이온농도,전기전도도,용존산소,탁도,총유기탄소,총질소,총인,클로로필-a,미생물_독성지수,Day sin,Day cos,Year sin,Year cos
0,-1.342174,0.333002,1.462974,0.158231,-0.286747,-0.760849,1.782384,-0.21750,-0.468079,0.337403,-1.999582e-12,1.414203,-0.003771,1.414155
1,-1.369209,0.333002,1.462974,0.158231,-0.305138,-0.760849,2.023195,-0.59571,-0.496459,0.275494,3.660228e-01,1.366016,-0.002757,1.414157
2,-1.355692,0.333002,1.462974,0.158231,-0.305138,-0.760849,2.167159,-0.59571,-0.496459,0.337403,7.071017e-01,1.224736,-0.001743,1.414159
3,-1.355692,0.333002,1.462974,0.158231,-0.286747,-0.760849,2.002255,-0.72178,-0.496459,2.937603,9.999929e-01,0.999993,-0.000730,1.414160
4,-1.355692,0.333002,1.462974,0.158231,-0.286747,-0.760849,1.926347,-0.59571,-0.496459,-1.519882,1.224736e+00,0.707102,0.000284,1.414160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,-1.463830,-0.490387,0.627070,1.154889,-0.194794,0.324071,0.002475,-0.72178,1.140110,0.089765,-1.366016e+00,0.366023,-0.008109,1.414137
35060,-1.477348,-0.490387,0.627070,1.154889,-0.194794,0.052841,0.023416,-0.84785,1.064430,0.337403,-1.224736e+00,0.707102,-0.007096,1.414142
35061,-1.490865,-0.490387,0.627070,1.154889,-0.194794,0.052841,0.054826,-0.72178,1.007671,0.585041,-9.999929e-01,0.999993,-0.006082,1.414147
35062,-1.490865,-0.490387,0.627070,1.154889,-0.194794,0.052841,0.015563,-0.72178,1.026590,0.461222,-7.071017e-01,1.224736,-0.005068,1.414151


In [31]:
wide_window.plot(plot_col='총유기탄소')
print('make_dataset_gain: dg.no = ', wide_window.dg.no)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

make_dataset_gain: dg.no =  2173


In [32]:
plt.figure(figsize=(9,10))
isnan = np.isnan(norm_data).astype(int)
data = ~isnan
n = data.shape[0]
seq_len = n//8
for i in range(8):
    plt.subplot(181+i)
    plt.imshow(data[i*seq_len:(i+1)*seq_len, 0:7], aspect='auto')
    plt.yticks([])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
plt.figure(figsize=(9,10))
n = wide_window.dg.data_m.shape[0]
train = n//8
for i in range(8):
    plt.subplot(181+i)
    plt.imshow(wide_window.dg.data_m[i*train:(i+1)*train, 0:7], aspect='auto')
    plt.yticks([])
#plt.imshow(wide_window.dg.data[0:100])
#plt.imshow(wide_window.dg.data_m[800:900], aspect='auto')
#print(wide_window.dg.data[0:50])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 컴파일 및 학습_GAIN

In [34]:
val_performance = {}
performance = {}

In [35]:
gain = GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False)
gain.compile(loss=GAIN.RMSE_loss)

In [36]:
MAX_EPOCHS = 2000

def compile_and_fit(model, window, patience=10):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  #model.compile(loss=tf.losses.MeanSquaredError(),
                #optimizer=tf.optimizers.Adam(),
                #metrics=[tf.metrics.MeanAbsoluteError()])
  model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

**모델 학습**

In [37]:
history = compile_and_fit(gain, wide_window, patience=MAX_EPOCHS//5)


val_performance['Gain'] = gain.evaluate(wide_window.val)
performance['Gain'] = gain.evaluate(wide_window.test, verbose=0)

gain.save(save_dir='save')

Epoch 1/2000
1/1 [==============================] - 1s 1s/step - gen_loss: 103.5437 - disc_loss: 0.7196 - rmse: 0.9240 - val_loss: 0.9105
Epoch 2/2000
1/1 [==============================] - 0s 172ms/step - gen_loss: 91.4530 - disc_loss: 0.5127 - rmse: 0.8615 - val_loss: 0.8921
Epoch 3/2000
1/1 [==============================] - 0s 170ms/step - gen_loss: 73.9385 - disc_loss: 0.5838 - rmse: 0.7851 - val_loss: 0.7517
Epoch 4/2000
1/1 [==============================] - 0s 168ms/step - gen_loss: 61.5937 - disc_loss: 0.3902 - rmse: 0.7070 - val_loss: 0.6990
Epoch 5/2000
1/1 [==============================] - 0s 170ms/step - gen_loss: 58.2711 - disc_loss: 0.4227 - rmse: 0.6622 - val_loss: 0.6397
Epoch 6/2000
1/1 [==============================] - 0s 167ms/step - gen_loss: 56.0725 - disc_loss: 0.4390 - rmse: 0.6677 - val_loss: 0.6294
Epoch 7/2000
1/1 [==============================] - 0s 173ms/step - gen_loss: 40.3701 - disc_loss: 0.4130 - rmse: 0.6012 - val_loss: 0.6134
Epoch 8/2000
1/1 [====

1/1 [==============================] - 0s 167ms/step - gen_loss: 5.6939 - disc_loss: 0.2348 - rmse: 0.2753 - val_loss: 0.5539
Epoch 118/2000
1/1 [==============================] - 0s 174ms/step - gen_loss: 7.4557 - disc_loss: 0.2442 - rmse: 0.3996 - val_loss: 0.3091
Epoch 119/2000
1/1 [==============================] - 0s 382ms/step - gen_loss: 8.4230 - disc_loss: 0.2422 - rmse: 0.4501 - val_loss: 0.4417
Epoch 120/2000
1/1 [==============================] - 0s 171ms/step - gen_loss: 6.7540 - disc_loss: 0.2352 - rmse: 0.4343 - val_loss: 0.2944
Epoch 121/2000
1/1 [==============================] - 0s 171ms/step - gen_loss: 6.8169 - disc_loss: 0.2518 - rmse: 0.3499 - val_loss: 0.3996
Epoch 122/2000
1/1 [==============================] - 0s 173ms/step - gen_loss: 6.7538 - disc_loss: 0.2439 - rmse: 0.4315 - val_loss: 0.3688
Epoch 123/2000
1/1 [==============================] - 0s 170ms/step - gen_loss: 6.4961 - disc_loss: 0.2402 - rmse: 0.3303 - val_loss: 0.3154
Epoch 124/2000
1/1 [========

1/1 [==============================] - 0s 169ms/step - gen_loss: 5.2926 - disc_loss: 0.1877 - rmse: 0.4928 - val_loss: 0.2947
Epoch 234/2000
1/1 [==============================] - 0s 167ms/step - gen_loss: 4.9385 - disc_loss: 0.1858 - rmse: 0.2829 - val_loss: 0.3026
Epoch 235/2000
1/1 [==============================] - 0s 165ms/step - gen_loss: 5.6728 - disc_loss: 0.1858 - rmse: 0.2927 - val_loss: 0.2902
Epoch 236/2000
1/1 [==============================] - 0s 166ms/step - gen_loss: 4.8739 - disc_loss: 0.1894 - rmse: 0.2811 - val_loss: 0.2907
Epoch 237/2000
1/1 [==============================] - 0s 166ms/step - gen_loss: 6.0546 - disc_loss: 0.1899 - rmse: 0.4980 - val_loss: 0.3063
Epoch 238/2000
1/1 [==============================] - 0s 361ms/step - gen_loss: 7.2911 - disc_loss: 0.1814 - rmse: 0.5516 - val_loss: 0.2798
Epoch 239/2000
1/1 [==============================] - 0s 170ms/step - gen_loss: 5.8569 - disc_loss: 0.1828 - rmse: 0.3081 - val_loss: 0.2897
Epoch 240/2000
1/1 [========

1/1 [==============================] - 0s 173ms/step - gen_loss: 5.9561 - disc_loss: 0.1958 - rmse: 0.3039 - val_loss: 0.3002
Epoch 350/2000
1/1 [==============================] - 0s 169ms/step - gen_loss: 5.1203 - disc_loss: 0.1869 - rmse: 0.2712 - val_loss: 0.2786
Epoch 351/2000
1/1 [==============================] - 0s 170ms/step - gen_loss: 5.1948 - disc_loss: 0.1804 - rmse: 0.2858 - val_loss: 0.3063
Epoch 352/2000
1/1 [==============================] - 0s 169ms/step - gen_loss: 5.7030 - disc_loss: 0.1858 - rmse: 0.2972 - val_loss: 0.2767
Epoch 353/2000
1/1 [==============================] - 0s 169ms/step - gen_loss: 5.0355 - disc_loss: 0.1868 - rmse: 0.2886 - val_loss: 0.3090
Epoch 354/2000
1/1 [==============================] - 0s 169ms/step - gen_loss: 4.5340 - disc_loss: 0.1884 - rmse: 0.2886 - val_loss: 0.2877
Epoch 355/2000
1/1 [==============================] - 0s 169ms/step - gen_loss: 5.7390 - disc_loss: 0.1881 - rmse: 0.2882 - val_loss: 0.3419
Epoch 356/2000
1/1 [========

1/1 [==============================] - 0s 172ms/step - gen_loss: 4.5090 - disc_loss: 0.1648 - rmse: 0.2805 - val_loss: 0.2811
Epoch 466/2000
1/1 [==============================] - 0s 172ms/step - gen_loss: 4.2269 - disc_loss: 0.1658 - rmse: 0.2962 - val_loss: 0.2971
Epoch 467/2000
1/1 [==============================] - 0s 179ms/step - gen_loss: 4.4713 - disc_loss: 0.1573 - rmse: 0.2958 - val_loss: 0.2750
Epoch 468/2000
1/1 [==============================] - 0s 176ms/step - gen_loss: 4.5926 - disc_loss: 0.1570 - rmse: 0.3176 - val_loss: 0.2798
Epoch 469/2000
1/1 [==============================] - 0s 173ms/step - gen_loss: 3.9608 - disc_loss: 0.1627 - rmse: 0.2792 - val_loss: 0.2927
Epoch 470/2000
1/1 [==============================] - 0s 169ms/step - gen_loss: 4.0993 - disc_loss: 0.1613 - rmse: 0.2724 - val_loss: 0.2949
Epoch 471/2000
1/1 [==============================] - 0s 170ms/step - gen_loss: 3.7980 - disc_loss: 0.1575 - rmse: 0.2834 - val_loss: 0.2813
Epoch 472/2000
1/1 [========

...

1/1 [==============================] - 0s 359ms/step - gen_loss: 3.3145 - disc_loss: 0.1502 - rmse: 0.2692 - val_loss: 0.2663
Epoch 698/2000
1/1 [==============================] - 0s 167ms/step - gen_loss: 3.2912 - disc_loss: 0.1543 - rmse: 0.2753 - val_loss: 0.2433
Epoch 699/2000
1/1 [==============================] - 0s 169ms/step - gen_loss: 3.2843 - disc_loss: 0.1558 - rmse: 0.2701 - val_loss: 0.2661
Epoch 700/2000
1/1 [==============================] - 0s 171ms/step - gen_loss: 3.0614 - disc_loss: 0.1486 - rmse: 0.2711 - val_loss: 0.2558
Epoch 701/2000
1/1 [==============================] - 0s 168ms/step - gen_loss: 4.7248 - disc_loss: 0.1503 - rmse: 0.3023 - val_loss: 0.2511
Epoch 702/2000
1/1 [==============================] - 0s 167ms/step - gen_loss: 3.7118 - disc_loss: 0.1585 - rmse: 0.2742 - val_loss: 0.2730
Epoch 703/2000
1/1 [==============================] - 0s 169ms/step - gen_loss: 3.4907 - disc_loss: 0.1511 - rmse: 0.2909 - val_loss: 0.2514
Epoch 704/2000
1/1 [========

1/1 [==============================] - 0s 168ms/step - gen_loss: 3.3431 - disc_loss: 0.1505 - rmse: 0.2476 - val_loss: 0.4567
Epoch 814/2000
1/1 [==============================] - 0s 171ms/step - gen_loss: 2.6203 - disc_loss: 0.1488 - rmse: 0.2402 - val_loss: 0.3799
Epoch 815/2000
1/1 [==============================] - 0s 355ms/step - gen_loss: 2.8856 - disc_loss: 0.1458 - rmse: 0.2360 - val_loss: 0.3788
Epoch 816/2000
1/1 [==============================] - 0s 175ms/step - gen_loss: 3.3379 - disc_loss: 0.1459 - rmse: 0.2856 - val_loss: 0.2723
Epoch 817/2000
1/1 [==============================] - 0s 173ms/step - gen_loss: 3.0801 - disc_loss: 0.1501 - rmse: 0.2690 - val_loss: 0.2621
Epoch 818/2000
1/1 [==============================] - 0s 166ms/step - gen_loss: 2.6776 - disc_loss: 0.1463 - rmse: 0.2664 - val_loss: 0.2854
Epoch 819/2000
1/1 [==============================] - 0s 170ms/step - gen_loss: 3.2328 - disc_loss: 0.1453 - rmse: 0.2901 - val_loss: 0.2571
Epoch 820/2000
1/1 [========

1/1 [==============================] - 0s 174ms/step - gen_loss: 3.8280 - disc_loss: 0.1368 - rmse: 0.2669 - val_loss: 0.2796
Epoch 930/2000
1/1 [==============================] - 0s 174ms/step - gen_loss: 2.8115 - disc_loss: 0.1354 - rmse: 0.2550 - val_loss: 0.2869
Epoch 931/2000
1/1 [==============================] - 0s 169ms/step - gen_loss: 2.7673 - disc_loss: 0.1366 - rmse: 0.2601 - val_loss: 0.2801
Epoch 932/2000
1/1 [==============================] - 0s 166ms/step - gen_loss: 2.7238 - disc_loss: 0.1311 - rmse: 0.2421 - val_loss: 0.2601
Epoch 933/2000
1/1 [==============================] - 0s 168ms/step - gen_loss: 3.1563 - disc_loss: 0.1397 - rmse: 0.2676 - val_loss: 0.2610
Epoch 934/2000
1/1 [==============================] - 0s 360ms/step - gen_loss: 2.9905 - disc_loss: 0.1396 - rmse: 0.4221 - val_loss: 0.2602
Epoch 935/2000
1/1 [==============================] - 0s 165ms/step - gen_loss: 2.6641 - disc_loss: 0.1419 - rmse: 0.4005 - val_loss: 0.2999
Epoch 936/2000
1/1 [========

Epoch 1045/2000
1/1 [==============================] - 0s 170ms/step - gen_loss: 2.4410 - disc_loss: 0.1318 - rmse: 0.3007 - val_loss: 0.2590
Epoch 1046/2000
1/1 [==============================] - 0s 165ms/step - gen_loss: 2.2559 - disc_loss: 0.1295 - rmse: 0.2792 - val_loss: 0.2611
Epoch 1047/2000
1/1 [==============================] - 0s 166ms/step - gen_loss: 2.2745 - disc_loss: 0.1307 - rmse: 0.2700 - val_loss: 0.2773
Epoch 1048/2000
1/1 [==============================] - 0s 166ms/step - gen_loss: 2.4023 - disc_loss: 0.1287 - rmse: 0.2807 - val_loss: 0.2983
Epoch 1049/2000
1/1 [==============================] - 0s 170ms/step - gen_loss: 2.2376 - disc_loss: 0.1304 - rmse: 0.2903 - val_loss: 0.2662
Epoch 1050/2000
1/1 [==============================] - 0s 170ms/step - gen_loss: 2.2871 - disc_loss: 0.1295 - rmse: 0.2496 - val_loss: 0.2691
Epoch 1051/2000
1/1 [==============================] - 0s 167ms/step - gen_loss: 2.2914 - disc_loss: 0.1297 - rmse: 0.2738 - val_loss: 0.2686
Epoch 

1/1 [==============================] - 0s 165ms/step - gen_loss: 2.1592 - disc_loss: 0.1404 - rmse: 0.2700 - val_loss: 0.2631
Epoch 1160/2000
1/1 [==============================] - 0s 168ms/step - gen_loss: 2.1434 - disc_loss: 0.1379 - rmse: 0.2730 - val_loss: 0.2694
Epoch 1161/2000
1/1 [==============================] - 0s 167ms/step - gen_loss: 2.1583 - disc_loss: 0.1379 - rmse: 0.2709 - val_loss: 0.2767
Epoch 1162/2000
1/1 [==============================] - 0s 167ms/step - gen_loss: 2.0612 - disc_loss: 0.1366 - rmse: 0.3327 - val_loss: 0.2701
Epoch 1163/2000
1/1 [==============================] - 0s 170ms/step - gen_loss: 2.5608 - disc_loss: 0.1320 - rmse: 0.2679 - val_loss: 0.2787
Epoch 1164/2000
1/1 [==============================] - 0s 168ms/step - gen_loss: 2.4803 - disc_loss: 0.1350 - rmse: 0.2928 - val_loss: 0.2804
Epoch 1165/2000
1/1 [==============================] - 0s 168ms/step - gen_loss: 2.2867 - disc_loss: 0.1303 - rmse: 0.2401 - val_loss: 0.2644
Epoch 1166/2000
1/1 [=

**학습 loss history 출력**

In [38]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax.plot(history.history['gen_loss'], label='gen_loss')
ax.plot(history.history['disc_loss'], label='disc_loss')
ax2.plot(history.history['rmse'], label='rmse', color='green')
ax2.plot(history.history['val_loss'], label='val_loss', color='red')
#plt.legend(history.history.keys(), loc='upper right')
#ax.legend(loc='upper center')
ax.legend(loc='upper center')
ax2.legend(loc='upper right')
ax.set_xlabel("epochs")
ax.set_ylabel("loss")
ax2.set_ylabel("rmse")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

성능 측정

In [39]:
gain.evaluate(wide_window.test.repeat(), steps=100)

100/100 [==============================] - 3s 32ms/step - loss: 0.2476


0.24763678014278412

샘플 prediction 출력

In [40]:
wide_window.plot(gain, plot_col='총유기탄소')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [41]:
norm_df = pd.concat(df,axis=0)

In [42]:
data = norm_df.to_numpy()

total_n = wide_window.dg.data.shape[0]
print(total_n)
unit_shape = wide_window.dg.shape[1:]
print(unit_shape)
#dim = np.prod(wide_window.dg.shape[1:]).astype(int)
dim = wide_window.dg.shape[1]
print(dim)
n = (total_n//dim)*dim

x = data[0:n].copy()
y_true = data[0:n].copy()

#x = interpolate(x, max_gap=3)

print('x.shape =', x.shape)
x_reshape = x.reshape((-1,)+unit_shape)
print('x_reshape.shape =', x_reshape.shape)
isnan = np.isnan(x_reshape)
print(isnan.sum())
print('y_true.shape=', y_true.shape)
isnan = np.isnan(y_true)
print(isnan.sum())

x_remain = data[-wide_window.dg.shape[1]:].copy()
x_remain_reshape = x_remain.reshape((-1,)+unit_shape)
x_remain_reshape.shape

70128
(120, 14)
120
x.shape = (70080, 14)
x_reshape.shape = (584, 120, 14)
136398
y_true.shape= (70080, 14)
136398


(1, 120, 14)

In [43]:
# zero loss is normal because there is no ground truth in the real dataset
gain.evaluate(x_reshape, y_true.reshape((-1,)+unit_shape))

19/19 [==============================] - 0s 6ms/step - loss: 0.0000e+00


0.0

In [44]:
y_pred = gain.predict(x_reshape)
y_remain_pred = gain.predict(x_remain_reshape)

In [45]:
print(total_n)
print(n)
print(total_n-n)

70128
70080
48


In [46]:
y_pred = y_pred.reshape(y_true.shape)
y_remain_pred = y_remain_pred.reshape(x_remain.shape)
print(y_pred.shape, y_remain_pred.shape)
y_pred = np.append(y_pred, y_remain_pred[-(total_n-n):], axis=0)
print(y_pred.shape)

(70080, 14) (120, 14)
(70128, 14)


In [47]:
y_pred[~np.isnan(data)] = np.nan

In [48]:
n = 8
plt.figure(figsize=(9,20))
for i in range(n):
    #plt.subplot('%d1%d'%(n,i))
    plt.subplot(811+i)
    plt.plot(x[:, i])
    plt.plot(y_pred[:, i])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [49]:
total_n = wide_window.dg.data.shape[0]
print(total_n)
unit_shape = wide_window.dg.shape[1:]
print('unit_shape=', unit_shape)
time_seq = unit_shape[0]
print(time_seq)
n = (total_n//time_seq)*time_seq
print('n=', n)

gans = []
oris = []
for i in range(len(df)):
    x = df[i].to_numpy()
    total_n = x.shape[0]
    n = (total_n//time_seq)*time_seq
    x = x[0:n]
    x_block = x.reshape((-1,)+unit_shape)
    y = gain.predict(x_block)
    y_gan = y.reshape(x.shape)
    
    # cut off sin, cos data
    if (i > 0):
        x = x[:, :-4]
        y_gan = y_gan[:, :-4]
    gans.append(y_gan)
    oris.append(x)
print(x.shape)
print(y_gan.shape)

70128
unit_shape= (120, 14)
120
n= 70080
(35040, 10)
(35040, 10)


## 예측 

In [50]:
total_no = oris[0].shape[0]
train_no = int(total_no*0.7)

train_slice = slice(0, train_no)
val_slice = slice(train_no, None)
test_slice = slice(0, None)

train_df = pd.DataFrame(gans[0][train_slice])
val_df = pd.DataFrame(gans[0][val_slice])
test_df = pd.DataFrame(gans[0][test_slice])

train_ori_df = pd.DataFrame(oris[0][train_slice])
val_ori_df = pd.DataFrame(oris[0][val_slice])
test_ori_df = pd.DataFrame(oris[0][test_slice])

num_features = train_df.shape[1]
out_num_features = num_features

In [51]:
ori = np.concatenate(oris, axis=1)
gan = np.concatenate(gans, axis=1)
print(oris[0].shape, gans[0].shape)
print(oris[1].shape, gans[1].shape)
print(ori.shape, gan.shape)

total_no = ori.shape[0]
train_no = int(total_no*0.7)

train_slice = slice(0, train_no)
val_slice = slice(train_no, None)
test_slice = slice(0, None)

train_df = pd.DataFrame(gan[train_slice])
val_df = pd.DataFrame(gan[val_slice])
test_df = pd.DataFrame(gan[test_slice])

train_ori_df = pd.DataFrame(ori[train_slice])
val_ori_df = pd.DataFrame(ori[val_slice])
test_ori_df = pd.DataFrame(ori[test_slice])

num_features = train_df.shape[1]
#out_num_features = oris[0].shape[1]-4
out_features = [6]
out_num_features = len(out_features)
out_num_features


(35040, 14) (35040, 14)
(35040, 10) (35040, 10)
(35040, 24) (35040, 24)


1

In [52]:
class WaterDataGenerator(keras.utils.Sequence):
    'Generates data for water'
    def __init__(self,
                 imputed_data,
                 ori_data = None,
                 batch_size=32,
                 input_width=24*7,
                 label_width=24*3,
                 shift=24*3,
                 skip_time = None,
                 shuffle = True,
                 out_features = None,
                 out_num_features = None,
                ):
        'Initialization'
        self.window_size = input_width+shift
        self.total_no = imputed_data.shape[0]
        self.data = imputed_data
        self.input_width = input_width
        self.label_width = label_width
        self.batch_size = batch_size
        self.input_shape = (batch_size, input_width, self.data.shape[1])
        self.out_num_features = out_num_features
        if out_features:
            self.out_features = out_features
        else:
            self.out_features = [i for i in range(out_num_features)]
        self.label_shape = (batch_size, label_width, self.out_num_features)
        if (skip_time):
            # TO-DO
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
        else:
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
            
        if shuffle:
            self.batch_idx = np.random.permutation(self.no)
        else:
            self.batch_idx = np.arange(0, self.no)
        self.batch_id = 0
        
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        #print('self.no =', self.no)
        #print('self.total_no =', self.total_no)
        #print('self.batch_id =', self.batch_id)
        # Sample batch
        label_width = self.label_width
        batch_idx = self.batch_idx
        
        x = np.empty((0, self.input_width, self.data.shape[1]))
        
        #y = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.label_width, self.out_num_features))
       
        #print(x.shape)
        #print(y.shape)
        
        
        for cnt in range(0, self.batch_size):
            i = self.batch_id
            self.batch_id += 1
            idx1 = self.data_idx[batch_idx[i]]
            idx2 = idx1 + self.input_width
            
            X = self.data[idx1:idx2].to_numpy()
            
            
            idx1 = self.data_idx[batch_idx[i]] + self.window_size - label_width
            idx2 = idx1 + label_width
            
            #Y = self.data[idx1:idx2,:,:out_num_features]
            Y = self.data.iloc[idx1:idx2, self.out_features].to_numpy()
            #Y = self.data[idx1:idx2]
            #print('Y.shape = ', Y.shape)
            #Y = Y.iloc[:,:out_num_features]
            
            self.batch_id %= self.no
            #print("x.shape=", x.shape)
            #print('X.shape=', X.shape)
            #print(type(x), type(X))
            x = np.append(x, [X], axis = 0)
            
            y = np.append(y, [Y], axis = 0)
            
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

**WaterDataGenerator 테스트**

In [53]:
OUT_STEPS = 24*3
wdg = WaterDataGenerator(train_df,
                         batch_size=128,
                         input_width = 24*7,
                         label_width = OUT_STEPS,
                         shift = OUT_STEPS,
                         out_num_features = out_num_features
                        )

In [54]:
train_df

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,-1.342174,0.333002,1.462974,0.158231,-0.286747,-0.760849,1.782384,-0.21750,-0.468079,0.337403,...,1.414204,-1.999581e-12,1.414155,-0.003771,0.038443,-0.078693,-1.531417,0.324341,0.703062,-0.218389
1,-1.369209,0.333002,1.462974,0.158231,-0.305138,-0.760849,2.023195,-0.59571,-0.496459,0.275494,...,1.366016,3.660228e-01,1.414157,-0.002757,0.086432,-0.078693,-1.531417,0.324341,0.703062,-0.218389
2,-1.355692,0.333002,1.462974,0.158231,-0.305138,-0.760849,2.167159,-0.59571,-0.496459,0.337403,...,1.224736,7.071018e-01,1.414159,-0.001743,0.102328,-0.078693,-1.531417,0.268971,0.703062,-0.218389
3,-1.355692,0.333002,1.462974,0.158231,-0.286747,-0.760849,2.002255,-0.72178,-0.496459,2.937603,...,0.999993,9.999928e-01,1.414160,-0.000730,0.049880,-0.078693,-1.544935,0.268971,0.703062,-0.218389
4,-1.355692,0.333002,1.462974,0.158231,-0.286747,-0.760849,1.926347,-0.59571,-0.496459,-1.519882,...,0.707102,1.224736e+00,1.414160,0.000284,0.081479,-0.078693,-1.544935,0.324341,0.703062,-0.218389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24523,0.293425,-0.902082,0.627070,-0.672317,0.007504,0.324071,0.913370,0.16071,-0.401860,-0.095963,...,0.366023,-1.366016e+00,0.412794,-1.352621,-0.046491,-0.696235,0.158251,-0.340098,-0.436807,0.595301
24524,0.279908,-1.107929,0.551079,-0.783057,-0.010886,0.324071,0.994513,0.03464,-0.439699,-0.343601,...,0.707102,-1.224736e+00,0.413763,-1.352324,-0.092766,-0.696235,0.158251,-0.340098,-0.436807,0.324071
24525,0.279908,-1.107929,0.551079,-0.783057,-0.010886,0.324071,1.004983,0.16071,-0.439699,-0.343601,...,0.999993,-9.999928e-01,0.414732,-1.352028,-0.019403,-0.696235,0.158251,-0.340098,-0.360816,0.324071
24526,0.279908,-1.107929,0.551079,-0.783057,-0.010886,0.324071,1.012835,0.16071,-0.439699,-0.095963,...,1.224736,-7.071018e-01,0.415701,-1.351730,-0.126992,-0.696235,0.144734,-0.340098,-0.360816,0.324071


In [55]:
it = iter(wdg)

In [56]:
x,y = next(it)
x.shape, y.shape

((128, 168, 24), (128, 72, 1))

In [57]:
def make_dataset_water(self, data):
  dg = WaterDataGenerator(
      data,
      batch_size = 128,
      input_width = self.input_width,
      label_width = self.label_width,
      shift = self.label_width,
      out_features = out_features,
      out_num_features = out_num_features,
  )
  #self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.input_shape,
        dg.label_shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  return ds

WindowGenerator.make_dataset = make_dataset_water

In [58]:
def plot2(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  plot_out_col_index = self.column_indices[plot_out_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
      label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
    else:
      label_col_index = plot_col_index
      label_out_col_index = plot_out_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_out_col_index],
                label='Labels', c='#2ca02c')
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_out_col_index],
                  marker=None, label='Predictions',
                  c='#ff7f0e')

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot2 = plot2

In [59]:

def hour_to_day_mean(array):
    time = 24
    return tf.reduce_mean(tf.reshape(array, [array.shape[0]//time, time]), 1)

def plot24(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  plot_out_col_index = self.column_indices[plot_out_col]
  max_n = min(max_subplots, len(inputs))

  if self.label_columns:
    label_col_index = self.label_columns_indices.get(plot_col, None)
    label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
  else:
    label_col_index = plot_col_index
    label_out_col_index = plot_out_col_index
    
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{df_all.columns[plot_col]} [normed]', fontproperties=fprop)
    
    input_temp = hour_to_day_mean(inputs[n, :, plot_col_index])
    input_temp = input_temp * train_std[plot_col] + train_mean[plot_col]
    
    plt.plot(
        hour_to_day_mean(self.input_indices),
        input_temp,
        label='Inputs', marker='.', zorder=-10)

    if label_col_index is None:
      continue
    
    label_temp = hour_to_day_mean(labels[n, :, label_out_col_index])
    label_temp = label_temp * train_std[plot_col] + train_mean[plot_col]
    
    plt.plot(
        hour_to_day_mean(self.label_indices),
        label_temp,
        label='Labels', marker='.', zorder=-10, c='#2ca02c')
    
    if model is not None:
      predictions = model(inputs)
    
      #predictions = predictions * train_std[plot_col] * train_mean[plot_col]
      predict_temp = hour_to_day_mean(predictions[n,:, label_out_col_index])
      predict_temp = predict_temp * train_std[plot_col] + train_mean[plot_col]
    
      plt.plot(
          hour_to_day_mean(self.label_indices),
          predict_temp,
          label='Predictions', marker='.', zorder=-10, c='#ff7f0e')

    if n == 0:
      plt.legend()

  plt.xlabel('Time [day]')

WindowGenerator.plot24 = plot24

In [91]:

cond = {
    "총인":[
        [-1000,0.02],
        [0.02,0.04],
        [0.04,0.10],
        [0.10,0.20],
        [0.20,0.30],
        [0.30,0.50],
        [0.50,1000],
    ],
    "총질소":[
        [-1000,0.2],
        [0.2,0.3],
        [0.3,0.4],
        [0.4,0.6],
        [0.6,1.0],
        [1.0,1.5],
        [1.5,1000],
    ],
    "수소이온농도":[
        [6.5 ,8.5],
        [6.5,8.5],
        [6.5,8.5],
        [6.5,8.5],
        [6.0,8.5],
        [6.0,8.5],
        [-1000,1000],
    ],
    "총유기탄소":[
        [-1000 ,2],
        [2,3],
        [3,4],
        [4,5],
        [5,6],
        [6,8],
        [8,1000],
    ],
    "용존산소":[
        [7.5 ,1000],
        [5,7.5],
        [5,7.5],
        [5,7.5],
        [2,5],
        [2,5],
        [-1000,2],
    ],
    "클로로필-a":[
        [-1000 ,5],
        [5,9],
        [9,14],
        [14,20],
        [20,42],
        [42,70],
        [70,1000],
    ],
    
}


def compa(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
  inputs, labels = self.example
  plot_out_col_index = self.column_indices[plot_out_col]

  if self.label_columns:
    label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
  else:
    label_out_col_index = plot_out_col_index
    
    
  if model is None:
    return
    
  print("column : " + df_all.columns[plot_col])
    
  count1 = 0
  count2 = 0
  for n in range(len(inputs)):
    predictions = model(inputs)
    pred_temp = hour_to_day_mean(predictions[n,:,label_out_col_index]) 
    pred_temp = pred_temp * train_std[plot_col] + train_mean[plot_col]
    label_temp = hour_to_day_mean(labels[n,:,label_out_col_index])
    label_temp = label_temp * train_std[plot_col] + train_mean[plot_col]
    
    level_label = 0 
    level_pred = 0
    
    for i in range(0,7,1):
      min, max = cond[df_all.columns[plot_col]][i]
      if min <= label_temp[2] and label_temp[2] < max:
        level_label = i
            #print(i, label_temp[2])
      if min <= pred_temp[2] and pred_temp[2] < max:
        level_pred = i
            
    if level_pred != level_label:
        count1 = count1 + 1
    count2 = count2 + 1 
        
  print(100 - count1*100/count2)

  return 100 - count1*100/count2
    

WindowGenerator.compa = compa

In [61]:
df_all.columns

Index(['수온', '수소이온농도', '전기전도도', '용존산소', '탁도', '총유기탄소', '총질소', '총인', '클로로필-a',
       '미생물_독성지수', 'Day sin', 'Day cos', 'Year sin', 'Year cos'],
      dtype='object')

In [95]:
out_features = [8]
#print(df_all.columns)

#multi_window.plot24(last_baseline, plot_col=out_features[0])
#multi_window.plot24(last_baseline, plot_col='총유기탄소')

multi_window.compa(last_baseline, plot_col=out_features[0])



column : 클로로필-a
58.59375


58.59375

In [87]:
multi_window.plot24(last_baseline, plot_col=out_features[0])

NameError: name 'last_baseline' is not defined

In [85]:
multi_window = WindowGenerator(input_width=24*7,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                               train_df=train_df,
                               val_df=val_df,
                               test_df=test_df,
                               )
multi_window

Total window size: 240
Input indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167]
Label indices: [168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185
 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203
 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221
 222 223 224 225 226 227 228 229 230 231 232 

In [88]:
MAX_EPOCHS = 400

def compile_and_fit(model, window, patience=1000):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])
#   model.compile(loss=tf.losses.MeanSquaredError(),
#                 optimizer=tf.optimizers.Adam(),
#                 metrics=[tf.metrics.MeanSquaredError()])
  #model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [89]:
def plot_history(history):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(history.history['loss'], label='loss')
    ax.plot(history.history['mean_absolute_error'], label='mae')
#     ax.plot(history.history['mean_squared_error'], label='mse')
    ax.plot(history.history['val_loss'], label='val_loss')
    ax.plot(history.history['val_mean_absolute_error'], label='val_mae')
#     ax.plot(history.history['val_mean_squared_error'], label='val_mse')
    #plt.legend(history.history.keys(), loc='upper right')
    #ax.legend(loc='upper center')
    ax.legend()
    ax.set_xlabel("epochs")
    ax.set_ylabel("loss")
    plt.show()

**Baseline model**

In [90]:
class MultiStepLastBaseline(tf.keras.Model):
  def call(self, inputs):
    #print(inputs[:, -1:, 0:1])
    #return tf.tile(inputs[:, -1:, :out_num_features], [1, OUT_STEPS, 1])
    return tf.tile(inputs[:, -1:, (out_features[0]):(out_features[0]+1)], [1, OUT_STEPS, 1])
    #return tf.tile(inputs[:, -1:, out_features[0]:(out_features[1]+1)], [1, OUT_STEPS, 1])

last_baseline = MultiStepLastBaseline()
last_baseline.compile(loss=tf.losses.MeanSquaredError(),
                      metrics=[tf.metrics.MeanAbsoluteError()])

multi_val_performance = {}
multi_performance = {}

multi_val_performance['BaseLine'] = last_baseline.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['BaseLine'] = last_baseline.evaluate(multi_window.test.repeat(-1), verbose=0, steps=100)
print('val performance =', multi_val_performance['BaseLine'])
print('test performance = ', multi_performance['BaseLine'])


100/100 [==============================] - 6s 62ms/step - loss: 0.0829 - mean_absolute_error: 0.1841
val performance = [0.08452838659286499, 0.18626023828983307]
test performance =  [0.19394321739673615, 0.24916408956050873]


In [97]:
_performance={}
_performance['base'] = multi_window.compa(last_baseline, plot_col=out_features[0])

column : 클로로필-a
58.59375


In [98]:
multi_window.plot24(last_baseline, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Linear**

In [99]:
import IPython
multi_linear_model = tf.keras.Sequential([
    # Take the last time-step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
    # Shape => [batch, 1, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

history = compile_and_fit(multi_linear_model, multi_window)

#IPython.display.clear_output()
multi_val_performance['Linear'] = multi_linear_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['Linear'] = multi_linear_model.evaluate(multi_window.test.repeat(-1), verbose=0, steps=100)
#multi_window.plot(multi_linear_model, plot_col=0)
print('val performance =', multi_val_performance['Linear'])
print('test performance = ', multi_performance['Linear'])

Epoch 1/400
1/1 [==============================] - 0s 450ms/step - loss: 0.6597 - mean_absolute_error: 0.5827 - val_loss: 0.2995 - val_mean_absolute_error: 0.4703
Epoch 2/400
1/1 [==============================] - 0s 151ms/step - loss: 0.6749 - mean_absolute_error: 0.5803 - val_loss: 0.3061 - val_mean_absolute_error: 0.4807
Epoch 3/400
1/1 [==============================] - 0s 155ms/step - loss: 0.4798 - mean_absolute_error: 0.5386 - val_loss: 0.2617 - val_mean_absolute_error: 0.4375
Epoch 4/400
1/1 [==============================] - 0s 157ms/step - loss: 0.7736 - mean_absolute_error: 0.5907 - val_loss: 0.3174 - val_mean_absolute_error: 0.5002
Epoch 5/400
1/1 [==============================] - 0s 155ms/step - loss: 0.4165 - mean_absolute_error: 0.5165 - val_loss: 0.3552 - val_mean_absolute_error: 0.5171
Epoch 6/400
1/1 [==============================] - 0s 155ms/step - loss: 0.7140 - mean_absolute_error: 0.5683 - val_loss: 0.2929 - val_mean_absolute_error: 0.4761
Epoch 7/400
1/1 [=====

1/1 [==============================] - 0s 157ms/step - loss: 0.6238 - mean_absolute_error: 0.5079 - val_loss: 0.2029 - val_mean_absolute_error: 0.3614
Epoch 52/400
1/1 [==============================] - 0s 156ms/step - loss: 0.5276 - mean_absolute_error: 0.4903 - val_loss: 0.2644 - val_mean_absolute_error: 0.4013
Epoch 53/400
1/1 [==============================] - 0s 157ms/step - loss: 0.3337 - mean_absolute_error: 0.4312 - val_loss: 0.1822 - val_mean_absolute_error: 0.3411
Epoch 54/400
1/1 [==============================] - 0s 155ms/step - loss: 0.2661 - mean_absolute_error: 0.3732 - val_loss: 0.2421 - val_mean_absolute_error: 0.3945
Epoch 55/400
1/1 [==============================] - 0s 157ms/step - loss: 0.4812 - mean_absolute_error: 0.4557 - val_loss: 0.2223 - val_mean_absolute_error: 0.3732
Epoch 56/400
1/1 [==============================] - 0s 153ms/step - loss: 0.2757 - mean_absolute_error: 0.3974 - val_loss: 0.2103 - val_mean_absolute_error: 0.3618
Epoch 57/400
1/1 [===========

1/1 [==============================] - 0s 152ms/step - loss: 0.3591 - mean_absolute_error: 0.3886 - val_loss: 0.2117 - val_mean_absolute_error: 0.3593
Epoch 102/400
1/1 [==============================] - 0s 158ms/step - loss: 0.3212 - mean_absolute_error: 0.3750 - val_loss: 0.3258 - val_mean_absolute_error: 0.4265
Epoch 103/400
1/1 [==============================] - 0s 162ms/step - loss: 0.3757 - mean_absolute_error: 0.4049 - val_loss: 0.3162 - val_mean_absolute_error: 0.4360
Epoch 104/400
1/1 [==============================] - 0s 164ms/step - loss: 0.5976 - mean_absolute_error: 0.4306 - val_loss: 0.2334 - val_mean_absolute_error: 0.3611
Epoch 105/400
1/1 [==============================] - 0s 157ms/step - loss: 0.2887 - mean_absolute_error: 0.3436 - val_loss: 0.2646 - val_mean_absolute_error: 0.3867
Epoch 106/400
1/1 [==============================] - 0s 156ms/step - loss: 0.2886 - mean_absolute_error: 0.3598 - val_loss: 0.2212 - val_mean_absolute_error: 0.3481
Epoch 107/400
1/1 [=====

Epoch 151/400
1/1 [==============================] - 0s 158ms/step - loss: 0.3518 - mean_absolute_error: 0.3907 - val_loss: 0.2276 - val_mean_absolute_error: 0.3666
Epoch 152/400
1/1 [==============================] - 0s 154ms/step - loss: 0.2847 - mean_absolute_error: 0.3817 - val_loss: 0.2399 - val_mean_absolute_error: 0.3785
Epoch 153/400
1/1 [==============================] - 0s 157ms/step - loss: 0.3463 - mean_absolute_error: 0.3736 - val_loss: 0.1984 - val_mean_absolute_error: 0.3387
Epoch 154/400
1/1 [==============================] - 0s 156ms/step - loss: 0.2627 - mean_absolute_error: 0.3530 - val_loss: 0.1781 - val_mean_absolute_error: 0.3290
Epoch 155/400
1/1 [==============================] - 0s 156ms/step - loss: 0.2662 - mean_absolute_error: 0.3541 - val_loss: 0.3168 - val_mean_absolute_error: 0.4245
Epoch 156/400
1/1 [==============================] - 0s 155ms/step - loss: 0.4032 - mean_absolute_error: 0.3974 - val_loss: 0.2099 - val_mean_absolute_error: 0.3585
Epoch 157/

Epoch 201/400
1/1 [==============================] - 0s 157ms/step - loss: 0.3643 - mean_absolute_error: 0.3755 - val_loss: 0.2359 - val_mean_absolute_error: 0.3728
Epoch 202/400
1/1 [==============================] - 0s 157ms/step - loss: 0.4438 - mean_absolute_error: 0.3561 - val_loss: 0.2217 - val_mean_absolute_error: 0.3848
Epoch 203/400
1/1 [==============================] - 0s 154ms/step - loss: 0.3072 - mean_absolute_error: 0.3366 - val_loss: 0.2125 - val_mean_absolute_error: 0.3511
Epoch 204/400
1/1 [==============================] - 0s 156ms/step - loss: 0.1915 - mean_absolute_error: 0.3136 - val_loss: 0.2712 - val_mean_absolute_error: 0.4128
Epoch 205/400
1/1 [==============================] - 0s 156ms/step - loss: 0.1862 - mean_absolute_error: 0.3026 - val_loss: 0.1901 - val_mean_absolute_error: 0.3481
Epoch 206/400
1/1 [==============================] - 0s 157ms/step - loss: 0.3422 - mean_absolute_error: 0.3414 - val_loss: 0.2172 - val_mean_absolute_error: 0.3659
Epoch 207/

Epoch 251/400
1/1 [==============================] - 0s 157ms/step - loss: 0.1722 - mean_absolute_error: 0.3083 - val_loss: 0.2078 - val_mean_absolute_error: 0.3496
Epoch 252/400
1/1 [==============================] - 0s 156ms/step - loss: 0.2620 - mean_absolute_error: 0.3297 - val_loss: 0.1794 - val_mean_absolute_error: 0.3282
Epoch 253/400
1/1 [==============================] - 0s 159ms/step - loss: 0.3059 - mean_absolute_error: 0.3267 - val_loss: 0.1964 - val_mean_absolute_error: 0.3341
Epoch 254/400
1/1 [==============================] - 0s 154ms/step - loss: 0.4014 - mean_absolute_error: 0.3440 - val_loss: 0.2134 - val_mean_absolute_error: 0.3554
Epoch 255/400
1/1 [==============================] - 0s 157ms/step - loss: 0.2177 - mean_absolute_error: 0.3351 - val_loss: 0.1956 - val_mean_absolute_error: 0.3432
Epoch 256/400
1/1 [==============================] - 0s 154ms/step - loss: 0.2885 - mean_absolute_error: 0.3500 - val_loss: 0.1521 - val_mean_absolute_error: 0.3030
Epoch 257/

Epoch 301/400
1/1 [==============================] - 0s 154ms/step - loss: 0.1778 - mean_absolute_error: 0.3140 - val_loss: 0.2275 - val_mean_absolute_error: 0.3646
Epoch 302/400
1/1 [==============================] - 0s 155ms/step - loss: 0.2584 - mean_absolute_error: 0.3336 - val_loss: 0.1633 - val_mean_absolute_error: 0.3211
Epoch 303/400
1/1 [==============================] - 0s 161ms/step - loss: 0.2490 - mean_absolute_error: 0.3206 - val_loss: 0.2073 - val_mean_absolute_error: 0.3632
Epoch 304/400
1/1 [==============================] - 0s 166ms/step - loss: 0.3470 - mean_absolute_error: 0.3722 - val_loss: 0.2302 - val_mean_absolute_error: 0.3701
Epoch 305/400
1/1 [==============================] - 0s 158ms/step - loss: 0.2429 - mean_absolute_error: 0.3294 - val_loss: 0.1991 - val_mean_absolute_error: 0.3520
Epoch 306/400
1/1 [==============================] - 0s 155ms/step - loss: 0.2661 - mean_absolute_error: 0.3321 - val_loss: 0.1825 - val_mean_absolute_error: 0.3380
Epoch 307/

Epoch 351/400
1/1 [==============================] - 0s 157ms/step - loss: 0.2040 - mean_absolute_error: 0.3127 - val_loss: 0.2305 - val_mean_absolute_error: 0.3776
Epoch 352/400
1/1 [==============================] - 0s 155ms/step - loss: 0.2665 - mean_absolute_error: 0.3279 - val_loss: 0.2296 - val_mean_absolute_error: 0.3669
Epoch 353/400
1/1 [==============================] - 0s 153ms/step - loss: 0.3673 - mean_absolute_error: 0.3692 - val_loss: 0.2179 - val_mean_absolute_error: 0.3777
Epoch 354/400
1/1 [==============================] - 0s 154ms/step - loss: 0.1719 - mean_absolute_error: 0.2933 - val_loss: 0.1881 - val_mean_absolute_error: 0.3351
Epoch 355/400
1/1 [==============================] - 0s 153ms/step - loss: 0.1610 - mean_absolute_error: 0.2908 - val_loss: 0.2014 - val_mean_absolute_error: 0.3593
Epoch 356/400
1/1 [==============================] - 0s 154ms/step - loss: 0.2030 - mean_absolute_error: 0.3316 - val_loss: 0.2073 - val_mean_absolute_error: 0.3605
Epoch 357/

100/100 [==============================] - 6s 62ms/step - loss: 0.1930 - mean_absolute_error: 0.3462
val performance = [0.19298075139522552, 0.34621763229370117]
test performance =  [0.22440363466739655, 0.3241894841194153]


In [100]:
multi_performance['Linear'] = multi_linear_model.evaluate(multi_window.test.repeat(-1), verbose=0, steps=100)

In [101]:
plot_history(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [102]:
_performance['Multi'] = multi_window.compa(multi_linear_model, plot_col=out_features[0])

column : 클로로필-a
51.5625


In [103]:
multi_window.plot24(multi_linear_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Elman_RNN**

In [104]:
elman_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.SimpleRNN(128, return_sequences=False),

    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),

    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [105]:
elman_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.SimpleRNN(128, return_sequences=True),
    tf.keras.layers.SimpleRNN(128, return_sequences=True),
    tf.keras.layers.SimpleRNN(128, return_sequences=True),
    tf.keras.layers.SimpleRNN(128, return_sequences=False),

    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),

    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [106]:
history = compile_and_fit(elman_model, multi_window)
multi_val_performance['ELMAN_RNN'] = elman_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['ELMAN_RNN'] = elman_model.evaluate(multi_window.test.repeat(-1), verbose=1, steps=100)
print('val performance =', multi_val_performance['ELMAN_RNN'])
print('test performance = ', multi_performance['ELMAN_RNN'])

Epoch 1/400
1/1 [==============================] - 2s 2s/step - loss: 0.7660 - mean_absolute_error: 0.5866 - val_loss: 0.3313 - val_mean_absolute_error: 0.4956
Epoch 2/400
1/1 [==============================] - 0s 371ms/step - loss: 0.5797 - mean_absolute_error: 0.5333 - val_loss: 0.2703 - val_mean_absolute_error: 0.4356
Epoch 3/400
1/1 [==============================] - 0s 369ms/step - loss: 0.9121 - mean_absolute_error: 0.5909 - val_loss: 0.3305 - val_mean_absolute_error: 0.4688
Epoch 4/400
1/1 [==============================] - 0s 363ms/step - loss: 0.5479 - mean_absolute_error: 0.5030 - val_loss: 0.2871 - val_mean_absolute_error: 0.4357
Epoch 5/400
1/1 [==============================] - 0s 373ms/step - loss: 0.6227 - mean_absolute_error: 0.5025 - val_loss: 0.2431 - val_mean_absolute_error: 0.3709
Epoch 6/400
1/1 [==============================] - 0s 383ms/step - loss: 0.4270 - mean_absolute_error: 0.4108 - val_loss: 0.2266 - val_mean_absolute_error: 0.3579
Epoch 7/400
1/1 [========

1/1 [==============================] - 0s 367ms/step - loss: 0.1231 - mean_absolute_error: 0.2559 - val_loss: 0.1082 - val_mean_absolute_error: 0.2394
Epoch 52/400
1/1 [==============================] - 0s 367ms/step - loss: 0.1919 - mean_absolute_error: 0.2914 - val_loss: 0.1029 - val_mean_absolute_error: 0.2505
Epoch 53/400
1/1 [==============================] - 0s 366ms/step - loss: 0.1772 - mean_absolute_error: 0.2673 - val_loss: 0.1491 - val_mean_absolute_error: 0.3031
Epoch 54/400
1/1 [==============================] - 0s 372ms/step - loss: 0.2911 - mean_absolute_error: 0.3437 - val_loss: 0.1744 - val_mean_absolute_error: 0.3307
Epoch 55/400
1/1 [==============================] - 0s 378ms/step - loss: 0.1243 - mean_absolute_error: 0.2761 - val_loss: 0.1315 - val_mean_absolute_error: 0.2693
Epoch 56/400
1/1 [==============================] - 0s 362ms/step - loss: 0.1021 - mean_absolute_error: 0.2359 - val_loss: 0.0851 - val_mean_absolute_error: 0.2266
Epoch 57/400
1/1 [===========

1/1 [==============================] - 0s 370ms/step - loss: 0.1157 - mean_absolute_error: 0.2219 - val_loss: 0.0951 - val_mean_absolute_error: 0.2451
Epoch 102/400
1/1 [==============================] - 0s 378ms/step - loss: 0.1465 - mean_absolute_error: 0.2469 - val_loss: 0.0915 - val_mean_absolute_error: 0.2405
Epoch 103/400
1/1 [==============================] - 0s 369ms/step - loss: 0.1309 - mean_absolute_error: 0.2414 - val_loss: 0.1006 - val_mean_absolute_error: 0.2493
Epoch 104/400
1/1 [==============================] - 0s 365ms/step - loss: 0.1206 - mean_absolute_error: 0.2415 - val_loss: 0.0897 - val_mean_absolute_error: 0.2289
Epoch 105/400
1/1 [==============================] - 0s 374ms/step - loss: 0.1381 - mean_absolute_error: 0.2289 - val_loss: 0.1251 - val_mean_absolute_error: 0.2777
Epoch 106/400
1/1 [==============================] - 0s 371ms/step - loss: 0.1117 - mean_absolute_error: 0.2226 - val_loss: 0.1685 - val_mean_absolute_error: 0.3265
Epoch 107/400
1/1 [=====

Epoch 151/400
1/1 [==============================] - 0s 372ms/step - loss: 0.1611 - mean_absolute_error: 0.2335 - val_loss: 0.1224 - val_mean_absolute_error: 0.2549
Epoch 152/400
1/1 [==============================] - 0s 361ms/step - loss: 0.1989 - mean_absolute_error: 0.2658 - val_loss: 0.1711 - val_mean_absolute_error: 0.2785
Epoch 153/400
1/1 [==============================] - 0s 369ms/step - loss: 0.1796 - mean_absolute_error: 0.2572 - val_loss: 0.0984 - val_mean_absolute_error: 0.2302
Epoch 154/400
1/1 [==============================] - 0s 368ms/step - loss: 0.0998 - mean_absolute_error: 0.2150 - val_loss: 0.1312 - val_mean_absolute_error: 0.2418
Epoch 155/400
1/1 [==============================] - 0s 372ms/step - loss: 0.0864 - mean_absolute_error: 0.2086 - val_loss: 0.1124 - val_mean_absolute_error: 0.2546
Epoch 156/400
1/1 [==============================] - 0s 366ms/step - loss: 0.2008 - mean_absolute_error: 0.2719 - val_loss: 0.1559 - val_mean_absolute_error: 0.2632
Epoch 157/

Epoch 201/400
1/1 [==============================] - 0s 364ms/step - loss: 0.1480 - mean_absolute_error: 0.2248 - val_loss: 0.1127 - val_mean_absolute_error: 0.2521
Epoch 202/400
1/1 [==============================] - 0s 367ms/step - loss: 0.2733 - mean_absolute_error: 0.2509 - val_loss: 0.1132 - val_mean_absolute_error: 0.2489
Epoch 203/400
1/1 [==============================] - 0s 368ms/step - loss: 0.1501 - mean_absolute_error: 0.2202 - val_loss: 0.1395 - val_mean_absolute_error: 0.2838
Epoch 204/400
1/1 [==============================] - 0s 374ms/step - loss: 0.0827 - mean_absolute_error: 0.2005 - val_loss: 0.1338 - val_mean_absolute_error: 0.2706
Epoch 205/400
1/1 [==============================] - 0s 376ms/step - loss: 0.0835 - mean_absolute_error: 0.2017 - val_loss: 0.1211 - val_mean_absolute_error: 0.2621
Epoch 206/400
1/1 [==============================] - 0s 371ms/step - loss: 0.0884 - mean_absolute_error: 0.1893 - val_loss: 0.1563 - val_mean_absolute_error: 0.2990
Epoch 207/

Epoch 251/400
1/1 [==============================] - 0s 361ms/step - loss: 0.1228 - mean_absolute_error: 0.2023 - val_loss: 0.1149 - val_mean_absolute_error: 0.2612
Epoch 252/400
1/1 [==============================] - 0s 386ms/step - loss: 0.1000 - mean_absolute_error: 0.2087 - val_loss: 0.1438 - val_mean_absolute_error: 0.3025
Epoch 253/400
1/1 [==============================] - 0s 365ms/step - loss: 0.1224 - mean_absolute_error: 0.2081 - val_loss: 0.1821 - val_mean_absolute_error: 0.3170
Epoch 254/400
1/1 [==============================] - 0s 371ms/step - loss: 0.1009 - mean_absolute_error: 0.2075 - val_loss: 0.1462 - val_mean_absolute_error: 0.2897
Epoch 255/400
1/1 [==============================] - 0s 363ms/step - loss: 0.1149 - mean_absolute_error: 0.2182 - val_loss: 0.1386 - val_mean_absolute_error: 0.2986
Epoch 256/400
1/1 [==============================] - 0s 362ms/step - loss: 0.0796 - mean_absolute_error: 0.2003 - val_loss: 0.1269 - val_mean_absolute_error: 0.2687
Epoch 257/

Epoch 301/400
1/1 [==============================] - 0s 370ms/step - loss: 0.1542 - mean_absolute_error: 0.2486 - val_loss: 0.0846 - val_mean_absolute_error: 0.2197
Epoch 302/400
1/1 [==============================] - 0s 363ms/step - loss: 0.1183 - mean_absolute_error: 0.2033 - val_loss: 0.0984 - val_mean_absolute_error: 0.2473
Epoch 303/400
1/1 [==============================] - 0s 359ms/step - loss: 0.1610 - mean_absolute_error: 0.2290 - val_loss: 0.1000 - val_mean_absolute_error: 0.2367
Epoch 304/400
1/1 [==============================] - 0s 363ms/step - loss: 0.1057 - mean_absolute_error: 0.2321 - val_loss: 0.1123 - val_mean_absolute_error: 0.2593
Epoch 305/400
1/1 [==============================] - 0s 374ms/step - loss: 0.0918 - mean_absolute_error: 0.2039 - val_loss: 0.1472 - val_mean_absolute_error: 0.2733
Epoch 306/400
1/1 [==============================] - 0s 365ms/step - loss: 0.1647 - mean_absolute_error: 0.2083 - val_loss: 0.0945 - val_mean_absolute_error: 0.2301
Epoch 307/

Epoch 351/400
1/1 [==============================] - 0s 365ms/step - loss: 0.0980 - mean_absolute_error: 0.2006 - val_loss: 0.1092 - val_mean_absolute_error: 0.2550
Epoch 352/400
1/1 [==============================] - 0s 370ms/step - loss: 0.1378 - mean_absolute_error: 0.2152 - val_loss: 0.1189 - val_mean_absolute_error: 0.2678
Epoch 353/400
1/1 [==============================] - 0s 365ms/step - loss: 0.0985 - mean_absolute_error: 0.1887 - val_loss: 0.1156 - val_mean_absolute_error: 0.2645
Epoch 354/400
1/1 [==============================] - 0s 361ms/step - loss: 0.0997 - mean_absolute_error: 0.1991 - val_loss: 0.1279 - val_mean_absolute_error: 0.2748
Epoch 355/400
1/1 [==============================] - 0s 368ms/step - loss: 0.0875 - mean_absolute_error: 0.1919 - val_loss: 0.1823 - val_mean_absolute_error: 0.3149
Epoch 356/400
1/1 [==============================] - 0s 378ms/step - loss: 0.0754 - mean_absolute_error: 0.1792 - val_loss: 0.1347 - val_mean_absolute_error: 0.2769
Epoch 357/

100/100 [==============================] - 12s 123ms/step - loss: 0.1093 - mean_absolute_error: 0.2138
val performance = [0.12276346981525421, 0.2579953968524933]
test performance =  [0.10925285518169403, 0.2137783318758011]


In [107]:
plot_history(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [108]:
_performance['Elman'] = multi_window.compa(elman_model, plot_col=out_features[0])

column : 클로로필-a
71.875


In [109]:
multi_window.plot24(elman_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**GRU**

In [110]:
gru_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
#     tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=False),
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [111]:
MAX_EPOCHS = 100

In [112]:
history = compile_and_fit(gru_model, multi_window)
multi_val_performance['GRU'] = gru_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['GRU'] = gru_model.evaluate(multi_window.test.repeat(-1), verbose=1, steps=100)
print('val performance =', multi_val_performance['GRU'])
print('test performance = ', multi_performance['GRU'])

Epoch 1/100
1/1 [==============================] - 1s 915ms/step - loss: 0.8652 - mean_absolute_error: 0.6336 - val_loss: 0.3756 - val_mean_absolute_error: 0.5397
Epoch 2/100
1/1 [==============================] - 0s 257ms/step - loss: 0.9776 - mean_absolute_error: 0.6243 - val_loss: 0.3088 - val_mean_absolute_error: 0.4784
Epoch 3/100
1/1 [==============================] - 0s 259ms/step - loss: 0.5322 - mean_absolute_error: 0.5336 - val_loss: 0.2822 - val_mean_absolute_error: 0.4634
Epoch 4/100
1/1 [==============================] - 0s 255ms/step - loss: 0.4436 - mean_absolute_error: 0.5024 - val_loss: 0.2971 - val_mean_absolute_error: 0.4791
Epoch 5/100
1/1 [==============================] - 0s 256ms/step - loss: 0.7585 - mean_absolute_error: 0.5607 - val_loss: 0.2919 - val_mean_absolute_error: 0.4603
Epoch 6/100
1/1 [==============================] - 0s 258ms/step - loss: 0.5799 - mean_absolute_error: 0.5428 - val_loss: 0.2511 - val_mean_absolute_error: 0.4214
Epoch 7/100
1/1 [=====

1/1 [==============================] - 0s 251ms/step - loss: 0.1345 - mean_absolute_error: 0.2600 - val_loss: 0.1028 - val_mean_absolute_error: 0.2363
Epoch 52/100
1/1 [==============================] - 0s 259ms/step - loss: 0.1323 - mean_absolute_error: 0.2288 - val_loss: 0.1201 - val_mean_absolute_error: 0.2672
Epoch 53/100
1/1 [==============================] - 0s 254ms/step - loss: 0.1065 - mean_absolute_error: 0.2315 - val_loss: 0.1017 - val_mean_absolute_error: 0.2421
Epoch 54/100
1/1 [==============================] - 0s 254ms/step - loss: 0.2183 - mean_absolute_error: 0.2624 - val_loss: 0.1175 - val_mean_absolute_error: 0.2639
Epoch 55/100
1/1 [==============================] - 0s 261ms/step - loss: 0.1613 - mean_absolute_error: 0.2500 - val_loss: 0.1034 - val_mean_absolute_error: 0.2468
Epoch 56/100
1/1 [==============================] - 0s 263ms/step - loss: 0.1881 - mean_absolute_error: 0.2644 - val_loss: 0.1131 - val_mean_absolute_error: 0.2468
Epoch 57/100
1/1 [===========

In [113]:
plot_history(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [114]:
_performance['GRU'] = multi_window.compa(gru_model, plot_col=out_features[0])

column : 클로로필-a
64.84375


In [115]:
multi_window.plot24(gru_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**LSTM**

In [116]:
multi_lstm_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units]
    # Adding more `lstm_units` just overfits more quickly.
    #tf.keras.layers.LSTM(32, return_sequences=False),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=False),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [131]:
history = compile_and_fit(multi_lstm_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.val)
multi_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['LSTM'])
print('test performance = ', multi_performance['LSTM'])

Epoch 1/400
1/1 [==============================] - 2s 2s/step - loss: 3.5012 - mean_absolute_error: 0.6438 - val_loss: 0.2107 - val_mean_absolute_error: 0.2574
Epoch 2/400
1/1 [==============================] - 0s 466ms/step - loss: 1.4801 - mean_absolute_error: 0.4697 - val_loss: 0.1842 - val_mean_absolute_error: 0.1982
Epoch 3/400
1/1 [==============================] - 0s 466ms/step - loss: 6.4112 - mean_absolute_error: 0.5780 - val_loss: 0.1757 - val_mean_absolute_error: 0.2020
Epoch 4/400
1/1 [==============================] - 0s 466ms/step - loss: 1.9451 - mean_absolute_error: 0.3907 - val_loss: 0.0620 - val_mean_absolute_error: 0.1879
Epoch 5/400
1/1 [==============================] - 0s 457ms/step - loss: 1.4112 - mean_absolute_error: 0.3544 - val_loss: 0.2228 - val_mean_absolute_error: 0.2322
Epoch 6/400
1/1 [==============================] - 0s 462ms/step - loss: 6.2268 - mean_absolute_error: 0.5175 - val_loss: 0.0766 - val_mean_absolute_error: 0.2084
Epoch 7/400
1/1 [========

1/1 [==============================] - 0s 468ms/step - loss: 0.9067 - mean_absolute_error: 0.2912 - val_loss: 0.0484 - val_mean_absolute_error: 0.1524
Epoch 52/400
1/1 [==============================] - 0s 466ms/step - loss: 6.8947 - mean_absolute_error: 0.5432 - val_loss: 0.1861 - val_mean_absolute_error: 0.2196
Epoch 53/400
1/1 [==============================] - 0s 488ms/step - loss: 2.9738 - mean_absolute_error: 0.3920 - val_loss: 0.1107 - val_mean_absolute_error: 0.2055
Epoch 54/400
1/1 [==============================] - 0s 469ms/step - loss: 0.5644 - mean_absolute_error: 0.3123 - val_loss: 0.1373 - val_mean_absolute_error: 0.2118
Epoch 55/400
1/1 [==============================] - 0s 467ms/step - loss: 0.9193 - mean_absolute_error: 0.2889 - val_loss: 0.0683 - val_mean_absolute_error: 0.1715
Epoch 56/400
1/1 [==============================] - 0s 461ms/step - loss: 1.0439 - mean_absolute_error: 0.2757 - val_loss: 0.0640 - val_mean_absolute_error: 0.1736
Epoch 57/400
1/1 [===========

1/1 [==============================] - 0s 465ms/step - loss: 0.4956 - mean_absolute_error: 0.2920 - val_loss: 0.1606 - val_mean_absolute_error: 0.2078
Epoch 102/400
1/1 [==============================] - 0s 464ms/step - loss: 0.8754 - mean_absolute_error: 0.2847 - val_loss: 0.1817 - val_mean_absolute_error: 0.2278
Epoch 103/400
1/1 [==============================] - 0s 463ms/step - loss: 0.7400 - mean_absolute_error: 0.3274 - val_loss: 0.1781 - val_mean_absolute_error: 0.2099
Epoch 104/400
1/1 [==============================] - 0s 470ms/step - loss: 0.2441 - mean_absolute_error: 0.2378 - val_loss: 0.0694 - val_mean_absolute_error: 0.1932
Epoch 105/400
1/1 [==============================] - 0s 457ms/step - loss: 3.4435 - mean_absolute_error: 0.3815 - val_loss: 0.1213 - val_mean_absolute_error: 0.2001
Epoch 106/400
1/1 [==============================] - 0s 459ms/step - loss: 0.4055 - mean_absolute_error: 0.2795 - val_loss: 0.0843 - val_mean_absolute_error: 0.1878
Epoch 107/400
1/1 [=====

Epoch 151/400
1/1 [==============================] - 0s 462ms/step - loss: 1.7865 - mean_absolute_error: 0.2420 - val_loss: 0.1256 - val_mean_absolute_error: 0.1844
Epoch 152/400
1/1 [==============================] - 0s 460ms/step - loss: 0.3108 - mean_absolute_error: 0.2046 - val_loss: 0.1626 - val_mean_absolute_error: 0.2121
Epoch 153/400
1/1 [==============================] - 0s 463ms/step - loss: 4.8011 - mean_absolute_error: 0.4410 - val_loss: 0.1123 - val_mean_absolute_error: 0.1856
Epoch 154/400
1/1 [==============================] - 0s 470ms/step - loss: 0.2111 - mean_absolute_error: 0.2089 - val_loss: 0.0807 - val_mean_absolute_error: 0.1918
Epoch 155/400
1/1 [==============================] - 0s 466ms/step - loss: 0.7393 - mean_absolute_error: 0.2557 - val_loss: 0.0603 - val_mean_absolute_error: 0.1696
Epoch 156/400
1/1 [==============================] - 0s 464ms/step - loss: 0.2670 - mean_absolute_error: 0.2025 - val_loss: 0.0952 - val_mean_absolute_error: 0.1841
Epoch 157/

Epoch 201/400
1/1 [==============================] - 0s 459ms/step - loss: 0.5619 - mean_absolute_error: 0.2341 - val_loss: 0.0720 - val_mean_absolute_error: 0.1663
Epoch 202/400
1/1 [==============================] - 0s 462ms/step - loss: 0.1946 - mean_absolute_error: 0.1789 - val_loss: 0.1038 - val_mean_absolute_error: 0.1662
Epoch 203/400
1/1 [==============================] - 0s 463ms/step - loss: 0.4040 - mean_absolute_error: 0.2248 - val_loss: 0.1576 - val_mean_absolute_error: 0.1773
Epoch 204/400
1/1 [==============================] - 0s 466ms/step - loss: 0.2483 - mean_absolute_error: 0.2033 - val_loss: 0.1184 - val_mean_absolute_error: 0.1664
Epoch 205/400
1/1 [==============================] - 0s 479ms/step - loss: 1.1380 - mean_absolute_error: 0.2355 - val_loss: 0.0741 - val_mean_absolute_error: 0.1590
Epoch 206/400
1/1 [==============================] - 0s 465ms/step - loss: 0.2450 - mean_absolute_error: 0.1918 - val_loss: 0.2089 - val_mean_absolute_error: 0.1878
Epoch 207/

Epoch 251/400
1/1 [==============================] - 0s 468ms/step - loss: 1.7694 - mean_absolute_error: 0.2979 - val_loss: 0.0651 - val_mean_absolute_error: 0.1837
Epoch 252/400
1/1 [==============================] - 0s 465ms/step - loss: 3.5122 - mean_absolute_error: 0.4194 - val_loss: 0.1203 - val_mean_absolute_error: 0.1987
Epoch 253/400
1/1 [==============================] - 0s 458ms/step - loss: 1.8212 - mean_absolute_error: 0.3232 - val_loss: 0.1816 - val_mean_absolute_error: 0.2636
Epoch 254/400
1/1 [==============================] - 0s 463ms/step - loss: 1.5433 - mean_absolute_error: 0.3122 - val_loss: 0.0922 - val_mean_absolute_error: 0.2119
Epoch 255/400
1/1 [==============================] - 0s 471ms/step - loss: 0.8192 - mean_absolute_error: 0.2913 - val_loss: 0.1707 - val_mean_absolute_error: 0.2092
Epoch 256/400
1/1 [==============================] - 0s 468ms/step - loss: 0.2294 - mean_absolute_error: 0.2171 - val_loss: 0.1272 - val_mean_absolute_error: 0.1980
Epoch 257/

Epoch 301/400
1/1 [==============================] - 0s 466ms/step - loss: 0.4157 - mean_absolute_error: 0.2157 - val_loss: 0.3462 - val_mean_absolute_error: 0.2309
Epoch 302/400
1/1 [==============================] - 0s 466ms/step - loss: 2.9369 - mean_absolute_error: 0.3420 - val_loss: 0.1123 - val_mean_absolute_error: 0.1651
Epoch 303/400
1/1 [==============================] - 0s 467ms/step - loss: 1.5994 - mean_absolute_error: 0.2947 - val_loss: 0.1186 - val_mean_absolute_error: 0.1712
Epoch 304/400
1/1 [==============================] - 0s 468ms/step - loss: 0.5382 - mean_absolute_error: 0.2222 - val_loss: 0.1519 - val_mean_absolute_error: 0.1696
Epoch 305/400
1/1 [==============================] - 0s 470ms/step - loss: 1.0043 - mean_absolute_error: 0.2910 - val_loss: 0.1604 - val_mean_absolute_error: 0.1598
Epoch 306/400
1/1 [==============================] - 0s 479ms/step - loss: 0.4041 - mean_absolute_error: 0.1643 - val_loss: 0.1449 - val_mean_absolute_error: 0.1615
Epoch 307/

Epoch 351/400
1/1 [==============================] - 0s 467ms/step - loss: 0.0728 - mean_absolute_error: 0.1284 - val_loss: 0.2666 - val_mean_absolute_error: 0.2317
Epoch 352/400
1/1 [==============================] - 0s 466ms/step - loss: 1.4791 - mean_absolute_error: 0.2564 - val_loss: 0.2540 - val_mean_absolute_error: 0.2466
Epoch 353/400
1/1 [==============================] - 0s 464ms/step - loss: 0.5489 - mean_absolute_error: 0.2369 - val_loss: 0.2426 - val_mean_absolute_error: 0.2186
Epoch 354/400
1/1 [==============================] - 0s 458ms/step - loss: 0.4678 - mean_absolute_error: 0.2198 - val_loss: 0.3074 - val_mean_absolute_error: 0.2123
Epoch 355/400
1/1 [==============================] - 0s 462ms/step - loss: 0.3203 - mean_absolute_error: 0.1724 - val_loss: 0.5593 - val_mean_absolute_error: 0.2475
Epoch 356/400
1/1 [==============================] - 0s 470ms/step - loss: 1.4489 - mean_absolute_error: 0.2881 - val_loss: 0.3801 - val_mean_absolute_error: 0.2110
Epoch 357/

1/1 [==============================] - 0s 177ms/step - loss: 0.5078 - mean_absolute_error: 0.2924
val performance = [0.5077596306800842, 0.29235827922821045]
test performance =  [0.8954910635948181, 0.2841479182243347]


In [118]:
plot_history(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [119]:
_performance['LSTM'] = multi_window.compa(multi_lstm_model, plot_col=out_features[0])

column : 클로로필-a
70.3125


In [120]:
multi_window.plot24(multi_lstm_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**CNN model**

In [121]:
CONV_WIDTH = 7
CONV_LAYER_NO = 1
multi_conv_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
    tf.keras.layers.Lambda(lambda x: x[:, -(CONV_WIDTH*CONV_LAYER_NO-CONV_LAYER_NO+1):, :]),
] + [
    # Shape => [batch, 1, conv_units]
    tf.keras.layers.Conv1D(1024, activation='relu', kernel_size=(CONV_WIDTH)) for i in range(CONV_LAYER_NO)
] + [
    # Shape => [batch, 1,  out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [122]:
MAX_EPOCHS = 400
history = compile_and_fit(multi_conv_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['Conv'])
print('test performance = ', multi_performance['Conv'])

Epoch 1/400
1/1 [==============================] - 0s 426ms/step - loss: 0.7057 - mean_absolute_error: 0.6050 - val_loss: 0.2972 - val_mean_absolute_error: 0.4736
Epoch 2/400
1/1 [==============================] - 0s 156ms/step - loss: 0.7088 - mean_absolute_error: 0.5528 - val_loss: 0.3108 - val_mean_absolute_error: 0.4692
Epoch 3/400
1/1 [==============================] - 0s 157ms/step - loss: 0.4963 - mean_absolute_error: 0.4962 - val_loss: 0.2595 - val_mean_absolute_error: 0.4140
Epoch 4/400
1/1 [==============================] - 0s 156ms/step - loss: 0.5268 - mean_absolute_error: 0.4847 - val_loss: 0.2328 - val_mean_absolute_error: 0.3804
Epoch 5/400
1/1 [==============================] - 0s 157ms/step - loss: 0.2852 - mean_absolute_error: 0.3976 - val_loss: 0.1826 - val_mean_absolute_error: 0.3065
Epoch 6/400
1/1 [==============================] - 0s 159ms/step - loss: 0.4679 - mean_absolute_error: 0.4636 - val_loss: 0.2419 - val_mean_absolute_error: 0.3862
Epoch 7/400
1/1 [=====

1/1 [==============================] - 0s 158ms/step - loss: 0.1078 - mean_absolute_error: 0.1988 - val_loss: 0.0809 - val_mean_absolute_error: 0.2030
Epoch 52/400
1/1 [==============================] - 0s 155ms/step - loss: 0.1667 - mean_absolute_error: 0.2552 - val_loss: 0.0889 - val_mean_absolute_error: 0.2107
Epoch 53/400
1/1 [==============================] - 0s 156ms/step - loss: 0.1117 - mean_absolute_error: 0.2211 - val_loss: 0.0729 - val_mean_absolute_error: 0.1899
Epoch 54/400
1/1 [==============================] - 0s 156ms/step - loss: 0.1680 - mean_absolute_error: 0.2539 - val_loss: 0.0986 - val_mean_absolute_error: 0.2284
Epoch 55/400
1/1 [==============================] - 0s 154ms/step - loss: 0.1960 - mean_absolute_error: 0.2461 - val_loss: 0.0817 - val_mean_absolute_error: 0.2044
Epoch 56/400
1/1 [==============================] - 0s 157ms/step - loss: 0.2089 - mean_absolute_error: 0.2506 - val_loss: 0.0668 - val_mean_absolute_error: 0.1918
Epoch 57/400
1/1 [===========

1/1 [==============================] - 0s 167ms/step - loss: 0.1229 - mean_absolute_error: 0.2212 - val_loss: 0.0694 - val_mean_absolute_error: 0.1914
Epoch 102/400
1/1 [==============================] - 0s 165ms/step - loss: 0.1332 - mean_absolute_error: 0.2516 - val_loss: 0.0636 - val_mean_absolute_error: 0.1793
Epoch 103/400
1/1 [==============================] - 0s 157ms/step - loss: 0.1871 - mean_absolute_error: 0.2574 - val_loss: 0.0540 - val_mean_absolute_error: 0.1634
Epoch 104/400
1/1 [==============================] - 0s 156ms/step - loss: 0.1596 - mean_absolute_error: 0.2231 - val_loss: 0.0615 - val_mean_absolute_error: 0.1732
Epoch 105/400
1/1 [==============================] - 0s 153ms/step - loss: 0.1118 - mean_absolute_error: 0.2080 - val_loss: 0.0545 - val_mean_absolute_error: 0.1737
Epoch 106/400
1/1 [==============================] - 0s 158ms/step - loss: 0.1793 - mean_absolute_error: 0.2289 - val_loss: 0.0625 - val_mean_absolute_error: 0.1768
Epoch 107/400
1/1 [=====

Epoch 151/400
1/1 [==============================] - 0s 163ms/step - loss: 0.1119 - mean_absolute_error: 0.2160 - val_loss: 0.0679 - val_mean_absolute_error: 0.1941
Epoch 152/400
1/1 [==============================] - 0s 156ms/step - loss: 0.1861 - mean_absolute_error: 0.2314 - val_loss: 0.0855 - val_mean_absolute_error: 0.2192
Epoch 153/400
1/1 [==============================] - 0s 158ms/step - loss: 0.1478 - mean_absolute_error: 0.2374 - val_loss: 0.0620 - val_mean_absolute_error: 0.1875
Epoch 154/400
1/1 [==============================] - 0s 158ms/step - loss: 0.1019 - mean_absolute_error: 0.2047 - val_loss: 0.0665 - val_mean_absolute_error: 0.1825
Epoch 155/400
1/1 [==============================] - 0s 163ms/step - loss: 0.1633 - mean_absolute_error: 0.2212 - val_loss: 0.0562 - val_mean_absolute_error: 0.1735
Epoch 156/400
1/1 [==============================] - 0s 167ms/step - loss: 0.1099 - mean_absolute_error: 0.2203 - val_loss: 0.0643 - val_mean_absolute_error: 0.1844
Epoch 157/

Epoch 201/400
1/1 [==============================] - 0s 155ms/step - loss: 0.1219 - mean_absolute_error: 0.2492 - val_loss: 0.0908 - val_mean_absolute_error: 0.2268
Epoch 202/400
1/1 [==============================] - 0s 152ms/step - loss: 0.1694 - mean_absolute_error: 0.2627 - val_loss: 0.0759 - val_mean_absolute_error: 0.2097
Epoch 203/400
1/1 [==============================] - 0s 156ms/step - loss: 0.1771 - mean_absolute_error: 0.2508 - val_loss: 0.0544 - val_mean_absolute_error: 0.1752
Epoch 204/400
1/1 [==============================] - 0s 154ms/step - loss: 0.1024 - mean_absolute_error: 0.2061 - val_loss: 0.0614 - val_mean_absolute_error: 0.1794
Epoch 205/400
1/1 [==============================] - 0s 156ms/step - loss: 0.1190 - mean_absolute_error: 0.2017 - val_loss: 0.0760 - val_mean_absolute_error: 0.2137
Epoch 206/400
1/1 [==============================] - 0s 154ms/step - loss: 0.1098 - mean_absolute_error: 0.2274 - val_loss: 0.0788 - val_mean_absolute_error: 0.2133
Epoch 207/

Epoch 251/400
1/1 [==============================] - 0s 155ms/step - loss: 0.1430 - mean_absolute_error: 0.2162 - val_loss: 0.0682 - val_mean_absolute_error: 0.1942
Epoch 252/400
1/1 [==============================] - 0s 155ms/step - loss: 0.1355 - mean_absolute_error: 0.2164 - val_loss: 0.0767 - val_mean_absolute_error: 0.2021
Epoch 253/400
1/1 [==============================] - 0s 153ms/step - loss: 0.0937 - mean_absolute_error: 0.1999 - val_loss: 0.0663 - val_mean_absolute_error: 0.1868
Epoch 254/400
1/1 [==============================] - 0s 161ms/step - loss: 0.0801 - mean_absolute_error: 0.1875 - val_loss: 0.0616 - val_mean_absolute_error: 0.1805
Epoch 255/400
1/1 [==============================] - 0s 153ms/step - loss: 0.1620 - mean_absolute_error: 0.2316 - val_loss: 0.0587 - val_mean_absolute_error: 0.1760
Epoch 256/400
1/1 [==============================] - 0s 156ms/step - loss: 0.1006 - mean_absolute_error: 0.2060 - val_loss: 0.0591 - val_mean_absolute_error: 0.1822
Epoch 257/

Epoch 301/400
1/1 [==============================] - 0s 163ms/step - loss: 0.1232 - mean_absolute_error: 0.2104 - val_loss: 0.0612 - val_mean_absolute_error: 0.1850
Epoch 302/400
1/1 [==============================] - 0s 156ms/step - loss: 0.0786 - mean_absolute_error: 0.1853 - val_loss: 0.0622 - val_mean_absolute_error: 0.1790
Epoch 303/400
1/1 [==============================] - 0s 159ms/step - loss: 0.1419 - mean_absolute_error: 0.2226 - val_loss: 0.0670 - val_mean_absolute_error: 0.1886
Epoch 304/400
1/1 [==============================] - 0s 157ms/step - loss: 0.0767 - mean_absolute_error: 0.1901 - val_loss: 0.0678 - val_mean_absolute_error: 0.1860
Epoch 305/400
1/1 [==============================] - 0s 155ms/step - loss: 0.2355 - mean_absolute_error: 0.2435 - val_loss: 0.0743 - val_mean_absolute_error: 0.1990
Epoch 306/400
1/1 [==============================] - 0s 157ms/step - loss: 0.1471 - mean_absolute_error: 0.2249 - val_loss: 0.0715 - val_mean_absolute_error: 0.1991
Epoch 307/

Epoch 351/400
1/1 [==============================] - 0s 156ms/step - loss: 0.1125 - mean_absolute_error: 0.2038 - val_loss: 0.0946 - val_mean_absolute_error: 0.2166
Epoch 352/400
1/1 [==============================] - 0s 160ms/step - loss: 0.1022 - mean_absolute_error: 0.2020 - val_loss: 0.0731 - val_mean_absolute_error: 0.2008
Epoch 353/400
1/1 [==============================] - 0s 162ms/step - loss: 0.0894 - mean_absolute_error: 0.2004 - val_loss: 0.0783 - val_mean_absolute_error: 0.2024
Epoch 354/400
1/1 [==============================] - 0s 164ms/step - loss: 0.1367 - mean_absolute_error: 0.2179 - val_loss: 0.0857 - val_mean_absolute_error: 0.2149
Epoch 355/400
1/1 [==============================] - 0s 166ms/step - loss: 0.1165 - mean_absolute_error: 0.2090 - val_loss: 0.0818 - val_mean_absolute_error: 0.2107
Epoch 356/400
1/1 [==============================] - 0s 164ms/step - loss: 0.1156 - mean_absolute_error: 0.1908 - val_loss: 0.0711 - val_mean_absolute_error: 0.1974
Epoch 357/

1/1 [==============================] - 0s 78ms/step - loss: 0.0838 - mean_absolute_error: 0.2167
val performance = [0.08384488523006439, 0.2167143076658249]
test performance =  [0.08772958815097809, 0.1997653841972351]


In [123]:
multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['Conv'])
print('test performance = ', multi_performance['Conv'])

1/1 [==============================] - 0s 78ms/step - loss: 0.0872 - mean_absolute_error: 0.2228
val performance = [0.08717723935842514, 0.2228425294160843]
test performance =  [0.14150360226631165, 0.22822587192058563]


In [124]:
plot_history(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [125]:
out_features = [4]
out_features[0]

4

In [126]:
multi_window.plot24(multi_conv_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [127]:
multi_window.plot24(multi_conv_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**performance**

In [128]:
x = np.arange(len(multi_performance))
width = 0.3


metric_name = 'mean_absolute_error'
metric_index = multi_conv_model.metrics_names.index('mean_absolute_error')
val_mae = [v[metric_index] for v in multi_val_performance.values()]
test_mae = [v[metric_index] for v in multi_performance.values()]

plt.figure()
plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=multi_performance.keys(),
           rotation=45)
plt.ylabel(f'MAE (average over all times and outputs)')
_ = plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [133]:
print(_performance)

{'base': 58.59375, 'Multi': 51.5625, 'Elman': 71.875, 'GRU': 64.84375, 'LSTM': 70.3125}


**Autoregressive model**

In [ ]:
class FeedBack(tf.keras.Model):
  def __init__(self, units, out_steps):
    super().__init__()
    self.out_steps = out_steps
    self.units = units
    self.lstm_cell = tf.keras.layers.LSTMCell(units)
    # Also wrap the LSTMCell in an RNN to simplify the `warmup` method.
    self.lstm_rnn = tf.keras.layers.RNN(self.lstm_cell, return_state=True)
    self.dense = tf.keras.layers.Dense(num_features)
    
feedback_model = FeedBack(units=32, out_steps=OUT_STEPS)

def warmup(self, inputs):
  # inputs.shape => (batch, time, features)
  # x.shape => (batch, lstm_units)
  x, *state = self.lstm_rnn(inputs)
  #print('x =', x)

  # predictions.shape => (batch, features)
  prediction = self.dense(x)
  return prediction, state

FeedBack.warmup = warmup

prediction, state = feedback_model.warmup(multi_window.example[0])
prediction.shape

def call(self, inputs, training=None):
  # Use a TensorArray to capture dynamically unrolled outputs.
  predictions = []
  # Initialize the lstm state
  prediction, state = self.warmup(inputs)

  # Insert the first prediction
  predictions.append(prediction)

  # Run the rest of the prediction steps
  for n in range(1, self.out_steps):
    # Use the last prediction as input.
    x = prediction
    # Execute one lstm step.
    x, state = self.lstm_cell(x, states=state,
                              training=training)
    # Convert the lstm output to a prediction.
    prediction = self.dense(x)
    # Add the prediction to the output
    predictions.append(prediction)

  # predictions.shape => (time, batch, features)
  predictions = tf.stack(predictions)
  # predictions.shape => (batch, time, features)
  predictions = tf.transpose(predictions, [1, 0, 2])
  predictions = tf.keras.layers.Lambda(lambda x: x[:, :, out_features[0]:(out_features[0]+len(out_features))])(predictions)
  return predictions

FeedBack.call = call

print('Output shape (batch, time, features): ', feedback_model(multi_window.example[0]).shape)

In [ ]:
history = compile_and_fit(feedback_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['AR LSTM'] = feedback_model.evaluate(multi_window.val)
multi_performance['AR LSTM'] = feedback_model.evaluate(multi_window.test, verbose=0)

print('val performance =', multi_val_performance['AR LSTM'])
print('test performance = ', multi_performance['AR LSTM'])

In [ ]:
plot_history(history)

In [ ]:
multi_window.plot24(feedback_model, plot_col=out_features[0])

**performance**

In [ ]:
x = np.arange(len(multi_performance))
width = 0.3


metric_name = 'mean_absolute_error'
metric_index = multi_conv_model.metrics_names.index('mean_absolute_error')
val_mae = [v[metric_index] for v in multi_val_performance.values()]
test_mae = [v[metric_index] for v in multi_performance.values()]

plt.figure()
plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=multi_performance.keys(),
           rotation=45)
plt.ylabel(f'MAE (average over all times and outputs)')
_ = plt.legend()
plt.show()